### 100 ETL sorusu PYTHON çıktıları 

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("sqlite-sakila.db")
cursor = conn.cursor()

 ### Veritabanı Tablolarını Pandas DataFrame’e Dönüştüren Fonksiyon

In [2]:
def read_table_as_dataframe(conn, table_name):
    """
    Verilen tabloyu sqlite3 bağlantısı üzerinden Pandas DataFrame olarak döndürür.
    read_sql() KULLANILMAZ.
    
    Parametreler:
        table_name (str): Alınacak tablo adı

    Döndürür:
        pd.DataFrame: Tablo verisi
    """

    #sql sorgusunu çalıştır
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)

    #tüm satırları çek
    rows = cursor.fetchall()

    #sutün adlarını al
    columns = [desc[0] for desc in cursor.description]

    #dataframe oluştur
    df = pd.DataFrame(rows,columns=columns)

    return df

In [3]:
category_df = read_table_as_dataframe(conn, "category")
film_category_df = read_table_as_dataframe(conn, "film_category")
film_df = read_table_as_dataframe(conn, "film")
inventory_df = read_table_as_dataframe(conn, "inventory")
rental_df = read_table_as_dataframe(conn, "rental")
payment_df = read_table_as_dataframe(conn, "payment")
actor_df = read_table_as_dataframe(conn, "actor")
language_df = read_table_as_dataframe(conn, "language")
film_actor_df = read_table_as_dataframe(conn, "film_actor")
staff_df = read_table_as_dataframe(conn, "staff")
store_df = read_table_as_dataframe(conn, "store")
address_df = read_table_as_dataframe(conn, "address")
city_df = read_table_as_dataframe(conn, "city")
country_df = read_table_as_dataframe(conn, "country")
customer_df = read_table_as_dataframe(conn, "customer")

### Soru 1: Tüm verileri göster

In [5]:
rental_df = read_table_as_dataframe(conn, "rental")
print(rental_df)

       rental_id              rental_date  inventory_id  customer_id  \
0              1  2005-05-24 22:53:30.000           367          130   
1              2  2005-05-24 22:54:33.000          1525          459   
2              3  2005-05-24 23:03:39.000          1711          408   
3              4  2005-05-24 23:04:41.000          2452          333   
4              5  2005-05-24 23:05:21.000          2079          222   
...          ...                      ...           ...          ...   
16039      16045  2005-08-23 22:25:26.000           772           14   
16040      16046  2005-08-23 22:26:47.000          4364           74   
16041      16047  2005-08-23 22:42:48.000          2088          114   
16042      16048  2005-08-23 22:43:07.000          2019          103   
16043      16049  2005-08-23 22:50:12.000          2666          393   

                   return_date  staff_id          last_update  
0      2005-05-26 22:04:30.000         1  2021-03-06 15:53:41  
1      

### Soru 2: Her filmdeki oyuncuları listele

In [14]:
# film + film_actor
merged_df1 = pd.merge(film_df,film_actor_df,on="film_id")
#+ actor
merged_df2 = pd.merge(merged_df1,actor_df,on="actor_id")
#istenen sutünlar
result_df = merged_df2[["title","first_name","last_name", "actor_id"]]
# actor_id'ye göre sırala
result_df_sorted = result_df.sort_values(by="actor_id").reset_index(drop=True)

# actor_id sütunu görünmesin diyorsan:
result_df_sorted = result_df_sorted[["title", "first_name", "last_name"]]

print(result_df_sorted)

                     title first_name last_name
0       COLOR PHILADELPHIA   PENELOPE   GUINESS
1          LANGUAGE COWBOY   PENELOPE   GUINESS
2              ANGELS LIFE   PENELOPE   GUINESS
3     ANACONDA CONFESSIONS   PENELOPE   GUINESS
4         OKLAHOMA JUMANJI   PENELOPE   GUINESS
...                    ...        ...       ...
5457    GALAXY SWEETHEARTS      THORA    TEMPLE
5458             RANDOM GO      THORA    TEMPLE
5459       TROJAN TOMORROW      THORA    TEMPLE
5460        WRONG BEHAVIOR      THORA    TEMPLE
5461           AFRICAN EGG      THORA    TEMPLE

[5462 rows x 3 columns]


### Soru 3: Her filmde kaç oyuncu oynadı?

In [16]:
# Önce film + film_actor birleştir
merged_df = pd.merge(film_df,film_actor_df,on="film_id")
# Sonra actor ile birleştir
merged_df = pd.merge(merged_df,actor_df,on="actor_id")
# Her film için kaç aktör var?
actor_count_df = merged_df.groupby("title").agg(actor_count=("actor_id","count")).reset_index()
print(actor_count_df)                                           

                 title  actor_count
0     ACADEMY DINOSAUR           10
1       ACE GOLDFINGER            4
2     ADAPTATION HOLES            5
3     AFFAIR PREJUDICE            5
4          AFRICAN EGG            5
..                 ...          ...
992     YOUNG LANGUAGE            5
993         YOUTH KICK            5
994       ZHIVAGO CORE            6
995  ZOOLANDER FICTION            5
996          ZORRO ARK            3

[997 rows x 2 columns]


### Soru 4: Her oyuncu kaç filmde oynadı?

In [18]:
merged_df = pd.merge(actor_df, film_actor_df, on="actor_id")
merged_df = pd.merge(merged_df, film_df, on="film_id")
# Her aktör için film sayısını bul
movie_count_df = merged_df.groupby(["first_name","last_name"]).agg(movie_count=("film_id","count")).reset_index()
print(movie_count_df)

    first_name  last_name  movie_count
0         ADAM      GRANT           18
1         ADAM     HOPPER           22
2           AL    GARLAND           26
3         ALAN   DREYFUSS           27
4       ALBERT  JOHANSSON           33
..         ...        ...          ...
194       WILL     WILSON           31
195    WILLIAM    HACKMAN           27
196      WOODY    HOFFMAN           31
197      WOODY      JOLIE           31
198       ZERO       CAGE           25

[199 rows x 3 columns]


### Soru 5: Envanterde olmayan filmler var mı ve varsa kaç tane?

In [20]:
# inventory'deki film_id'ler
inventory_film_ids = inventory_df["film_id"].unique()

# Envanterde olmayan filmleri filtrele
not_in_inventory_df = film_df[~film_df["film_id"].isin(inventory_film_ids)]
# Sayısını al
not_in_inventory_count = len(not_in_inventory_df)
# Sonucu bir DataFrame olarak göster 
result_df = pd.DataFrame({"not_in_inventory":[not_in_inventory_count]})
print(result_df)

   not_in_inventory
0                42


### Soru 6: Kiralanabilir olan her filmin kaç kez kiralandığını ve toplam gelirlerini getirin

In [22]:
# film + inventory
merged1 = pd.merge(film_df, inventory_df, on="film_id")

# + rental
merged2 = pd.merge(merged1, rental_df, on="inventory_id")

# + payment

merged3 = pd.merge(merged2, payment_df, on="rental_id", suffixes=('_left', '_right'))
#Kullanılmayan sütunları kaldır:merged3 için alternatif yol 
# merged2 = merged2.drop(columns=["last_update"]) payment_df = payment_df.drop(columns=["last_update"])
# merged3 = pd.merge(merged2, payment_df, on="rental_id")
result_df = merged3.groupby("title").agg(
    rental_count=("rental_id", "count"),
    total_revenue=("amount", "sum")
).reset_index()

print(result_df)


                 title  rental_count  total_revenue
0     ACADEMY DINOSAUR            23          36.77
1       ACE GOLDFINGER             7          52.93
2     ADAPTATION HOLES            12          37.88
3     AFFAIR PREJUDICE            23          91.77
4          AFRICAN EGG            12          51.88
..                 ...           ...            ...
953     YOUNG LANGUAGE             7           6.93
954         YOUTH KICK             6          16.94
955       ZHIVAGO CORE             9          14.91
956  ZOOLANDER FICTION            17          73.83
957          ZORRO ARK            31         214.69

[958 rows x 3 columns]


### Soru 7: Envanterde olmayan filmlerin kira oranlarını getirin

In [24]:
# inventory'de olan film_id'ler
inventory_film_ids = inventory_df["film_id"].unique()
# inventory'de olmayanları filtrele
not_in_inventory_df = film_df[~film_df["film_id"].isin(inventory_film_ids)]
# Sadece title ve rental_rate sütunlarını al
result_df = not_in_inventory_df[["title","rental_rate"]]
print(result_df)

                      title  rental_rate
13           ALICE FANTASIA         0.99
32              APOLLO TEEN         2.99
35           ARGONAUTS TOWN         0.99
37            ARK RIDGEMONT         0.99
40     ARSENIC INDEPENDENCE         0.99
86        BOONDOCK BALLROOM         0.99
107           BUTCH PANTHER         0.99
127           CATCH AMISTAD         0.99
143     CHINATOWN GLADIATOR         4.99
147          CHOCOLATE DUCK         2.99
170    COMMANDMENTS EXPRESS         4.99
191        CROSSING DIVORCE         4.99
194         CROWDS TELEMARK         4.99
197        CRYSTAL BREAKING         2.99
216              DAZED PUNK         4.99
220  DELIVERANCE MULHOLLAND         0.99
317       FIREHOUSE VIETNAM         0.99
324           FLOATS GARDEN         2.99
331   FRANKENSTEIN STRANGER         0.99
358      GLADIATOR WESTWARD         4.99
385               GUMP DATE         4.99
403           HATE HANDICAP         0.99
418             HOCUS FRIDA         2.99
494        KENTU

### Soru 8: Birden fazla DVD'yi iade etmeyen kaç müşteri var?

In [26]:
# 1. return_date IS NULL (yani aktif kiralamalar)
active_rentals_df = rental_df[rental_df['return_date'].isna()]

# 2. customer_id'ye göre rental say
rental_counts = (
    active_rentals_df
    .groupby('customer_id')
    .agg(rental_count=('rental_id', 'count'))
    .reset_index()
)

# 3. rental_count > 1 olan müşterileri al
multi_active_rentals = rental_counts[rental_counts['rental_count'] > 1]

# 4. Her biri için "1" değeri olan bir sütun ekle
result_df = pd.DataFrame({'customer_count': [1] * len(multi_active_rentals)})

print(result_df)

    customer_count
0                1
1                1
2                1
3                1
4                1
5                1
6                1
7                1
8                1
9                1
10               1
11               1
12               1
13               1
14               1
15               1
16               1
17               1
18               1
19               1
20               1
21               1
22               1


### Soru 9: Her müşteri kaç film kiraladı?

In [28]:
# 1. customer_id ile birleştir
merged_df = pd.merge(customer_df,rental_df,on="customer_id")
# 2. first_name + last_name’e göre grupla ve say
result_df = merged_df.groupby(["first_name","last_name"]).agg(rental_count=("rental_id","count")).reset_index()
print(result_df)                              

    first_name last_name  rental_count
0        AARON     SELBY            24
1         ADAM     GOOCH            22
2       ADRIAN     CLARY            19
3        AGNES    BISHOP            23
4         ALAN      KAHN            26
..         ...       ...           ...
594     WILLIE   MARKHAM            25
595      WILMA  RICHARDS            20
596    YOLANDA    WEAVER            27
597     YVONNE   WATKINS            21
598    ZACHARY      HITE            31

[599 rows x 3 columns]


### Soru 10: Türlerine göre en çok kiralanan filmler ve bunlara ne kadar ödendi?

In [30]:
# category + film_category
merged1 = pd.merge(category_df,film_category_df,on="category_id")

# + film
merged2 = pd.merge(merged1, film_df, on="film_id")

# + inventory
merged3 = pd.merge(merged2, inventory_df, on="film_id", suffixes=('_film', '_inventory'))

# + rental
merged4 = pd.merge(merged3, rental_df, on="inventory_id")

# + payment
merged5 = pd.merge(merged4, payment_df, on="rental_id",suffixes=('_rental', '_payment'))

# Mevcut kodunla aynı şekilde gruplama ve toplama
result_df = merged5.groupby(["name", "title"]).agg(
    rental_count=("rental_id", "count"),
    total_paid=("amount", "sum")
).reset_index()

# Index'e göre sıralamak için geçici bir sütun ekleyelim
result_df['index_col'] = result_df.index

# rental_count'a göre artan, eşitse index'e göre artan sırala
result_df = result_df.sort_values(by=["rental_count", "index_col"], ascending=[False, True])

# Geçici sütunu kaldır
result_df = result_df.drop(columns=["index_col"])

print(result_df)

            name               title  rental_count  total_paid
909       Travel  BUCKET BROTHERHOOD            34      180.66
536      Foreign    ROCKETEER MOTHER            33      116.67
93     Animation      JUGGLER HARDLY            32       96.68
568        Games      FORWARD TEMPLE            32      128.68
572        Games      GRIT CLOCKWORK            32      110.68
..           ...                 ...           ...         ...
872       Sports  MUSSOLINI SPOILERS             5       15.95
951       Travel      TRAFFIC HOBBIT             5       24.95
315  Documentary      HARDLY ROBBERS             4       15.96
525      Foreign         MIXED DOORS             4       15.96
656       Horror         TRAIN BUNCH             4       24.96

[958 rows x 4 columns]


### Soru 11: Tür ve Tarihe Göre Kiralama Sayısı ve Gelir

In [19]:
# 1. category + film_category
df = pd.merge(category_df, film_category_df, on="category_id")

# 2. + film
df = pd.merge(df, film_df, on="film_id")

# 3. + inventory
df = pd.merge(df, inventory_df, on="film_id", suffixes=('_film', '_inventory'))

# 4. + rental
df = pd.merge(df, rental_df, on="inventory_id")

# 5. + payment
df = pd.merge(df, payment_df, on="rental_id", suffixes=('_left', '_right'))

# rental_date sadece tarih olarak al
df["rental_date"] = pd.to_datetime(df["rental_date"]).dt.date

# Gerekli sütunları seç
df = df[["name", "rental_date", "rental_id", "amount"]]

# Kolon isimlerini SQL çıktısına göre düzelt
df = df.rename(columns={
    "name": "genre",
    "amount": "SUM(payment.amount)"
})

# rental_id başına 1 olduğu için yeni sütun ekle
df["rental_count"] = 1

# Sıralama
df = df.sort_values(by=["rental_date", "genre"])
df = df[["genre", "rental_date", "rental_count", "SUM(payment.amount)"]]
print(df)

           genre rental_date  rental_count  SUM(payment.amount)
1742   Animation  2005-05-24             1                 4.99
2617    Children  2005-05-24             1                 3.99
2715    Children  2005-05-24             1                 6.99
4727      Comedy  2005-05-24             1                 0.99
7336      Family  2005-05-24             1                 2.99
...          ...         ...           ...                  ...
15643     Travel  2006-02-14             1                 0.99
15774     Travel  2006-02-14             1                 4.99
15829     Travel  2006-02-14             1                 4.99
15943     Travel  2006-02-14             1                 0.00
15974     Travel  2006-02-14             1                 2.99

[16044 rows x 4 columns]


### Soru 12: Kiralanabilir Filmler İçin Türlerine Göre Her Filmin Kaç Kez Kiralandığı

In [38]:
# category + film_category
merged1 = pd.merge(category_df, film_category_df, on="category_id")

# + film
merged2 = pd.merge(merged1, film_df, on="film_id")

# + inventory
merged3 = pd.merge(merged2, inventory_df, on="film_id", suffixes=('_film', '_inventory'))

# + rental
merged4 = pd.merge(merged3, rental_df, on="inventory_id")

result_df = merged4.groupby(["name", "title"]).agg(
    rental_count=("rental_id", "count")
).reset_index()
result_df = result_df.rename(columns={"name": "genre"})
print(result_df)                                                   

      genre                title  rental_count
0    Action         AMADEUS HOLY            21
1    Action      AMERICAN CIRCUS            22
2    Action   ANTITRUST TOMATOES            10
3    Action  BAREFOOT MANCHURIAN            18
4    Action         BERETS AGENT            21
..      ...                  ...           ...
953  Travel  VALENTINE VANISHING            12
954  Travel          WINDOW SIDE            12
955  Travel        WOLVES DESIRE            21
956  Travel        WORKER TARZAN            15
957  Travel  WORKING MICROCOSMOS            25

[958 rows x 3 columns]


### Soru 13: En çok rafta bekleyen filmler?

In [17]:

film_df = read_table_as_dataframe(conn, "film")[["film_id", "title"]]
inventory_df = read_table_as_dataframe(conn, "inventory")[["inventory_id", "film_id"]]
rental_df = read_table_as_dataframe(conn, "rental")[["rental_id", "inventory_id", "rental_date", "return_date"]]

merged1 = pd.merge(film_df, inventory_df, on="film_id")
merged2 = pd.merge(merged1, rental_df, on="inventory_id")

unreturned = merged2[merged2["return_date"].isna()].copy()

unreturned["days_on_shelf"] = (pd.to_datetime("now") - pd.to_datetime(unreturned["rental_date"])).dt.days

result_df = unreturned[["title", "days_on_shelf"]].sort_values(by="days_on_shelf", ascending=False)

print(result_df)

                    title  days_on_shelf
16       ACADEMY DINOSAUR           7249
24         ACE GOLDFINGER           7072
61       AFFAIR PREJUDICE           7072
76            AFRICAN EGG           7072
224           ALI FOREVER           7072
...                   ...            ...
15615         WILD APOLLO           7072
15658         WINDOW SIDE           7072
15745        WOMEN DORADO           7072
15887  WORLD LEATHERNECKS           7072
15995        ZHIVAGO CORE           7072

[183 rows x 2 columns]


### Soru 14: Geç, Erken ve Zamanında İade Edilen Kiralanmış Filmler(*)

In [54]:
# 1. rental + inventory join
merged1 = pd.merge(rental_df, inventory_df, on='inventory_id')

# 2. + film join (film_id üzerinden)
merged2 = pd.merge(merged1, film_df[['film_id', 'rental_duration']], on='film_id')

# 3. tarih sütunlarını datetime'a çevir
merged2['rental_date'] = pd.to_datetime(merged2['rental_date'])
merged2['return_date'] = pd.to_datetime(merged2['return_date'])
merged2 = merged2[merged2['return_date'].notna()]  # NULL return_date çıkar

# 4. due_date hesapla
merged2['due_date'] = merged2['rental_date'] + pd.to_timedelta(merged2['rental_duration'], unit='d')

# 5. koşullar için boolean maskeler
late_mask = merged2['return_date'] > merged2['due_date']
early_mask = merged2['return_date'] < merged2['due_date']
on_time_mask = merged2['return_date'] == merged2['due_date']

# 6. toplamları hesapla
late_returns = late_mask.sum()
early_returns = early_mask.sum()
on_time_returns = on_time_mask.sum()

# 7. sonuçları DataFrame olarak yazdır
result_df = pd.DataFrame({
    'late_returns': [late_returns],
    'early_returns': [early_returns],
    'on_time_returns': [on_time_returns]
})

print(result_df)

   late_returns  early_returns  on_time_returns
0          8121           7738                2


### Soru 15: Hangi müşteri en çok DVD kiralamış?

In [56]:
merged_df = pd.merge(customer_df, rental_df, on="customer_id", how="inner")

# İsim ve soyadına göre gruplama yap, rental_id say
rental_counts = merged_df.groupby(['first_name', 'last_name'])['rental_id'].count().reset_index()

# rental_count adını ver
rental_counts = rental_counts.rename(columns={'rental_id': 'rental_count'})

# rental_count'a göre azalan sıralama yap
rental_counts = rental_counts.sort_values(by='rental_count', ascending=False)

# En çok kira yapan müşteri (ilk satır)
top_customer = rental_counts.head(1)

print(top_customer)

    first_name last_name  rental_count
175    ELEANOR      HUNT            46


### Soru 16: En popüler film kategorisi nedir?

In [58]:
# 1. category + film_category
merged1 = pd.merge(category_df, film_category_df, on="category_id")

# 2. + film
merged2 = pd.merge(merged1, film_df, on="film_id")

# 3. + inventory
merged3 = pd.merge(merged2, inventory_df, on="film_id", suffixes=('_film', '_inventory'))

# 4. + rental
merged4 = pd.merge(merged3, rental_df, on="inventory_id")

# 5. category name'e göre gruplama ve rental_id sayımı
result_df = merged4.groupby("name")["rental_id"].count().reset_index(name="rental_count")

# 6. rental_count'a göre azalan sıralama
result_df = result_df.sort_values(by="rental_count", ascending=False)

# 7. sonucu yazdır
print(result_df)

           name  rental_count
14       Sports          1179
1     Animation          1166
0        Action          1112
13       Sci-Fi          1101
7        Family          1096
6         Drama          1060
5   Documentary          1050
8       Foreign          1033
9         Games           969
2      Children           945
4        Comedy           941
12          New           940
3      Classics           939
10       Horror           846
15       Travel           837
11        Music           830


### Soru 17: Hangi çalışan en çok kiralama işlemi gerçekleştirmiş?


In [60]:
# staff ve rental tablolarını birleştir
merged_df = pd.merge(staff_df, rental_df, on="staff_id")

# ad ve soyada göre gruplama, rental_id sayımı
rental_counts = merged_df.groupby(["first_name", "last_name"])["rental_id"].count().reset_index(name="rental_count")

# rental_count'a göre azalan sırala
rental_counts = rental_counts.sort_values(by="rental_count", ascending=False)

# en çok kiralama yapan personeli al
top_staff = rental_counts.head(1)

print(top_staff)

  first_name last_name  rental_count
1       Mike   Hillyer          8040


### Soru 18: En çok geliri hangi film getirmiş?

In [62]:
merged1 = pd.merge(film_df, inventory_df, on="film_id")
merged2 = pd.merge(merged1, rental_df, on="inventory_id")
merged3 = pd.merge(merged2, payment_df, on="rental_id",suffixes=("_inv","_rent"))

revenue_df = merged3.groupby("title")["amount"].sum().reset_index(name="total_revenue")
revenue_df = revenue_df.sort_values(by="total_revenue", ascending=False)
top_film = revenue_df.head(1)
print(top_film)

                title  total_revenue
841  TELEGRAPH VOYAGE         231.73


### Soru 19: Her müşteri için toplam harcama miktarını bulun

In [64]:
# 1. customer + rental
merged1 = pd.merge(customer_df, rental_df, on="customer_id")

# 2. + payment
merged2 = pd.merge(merged1, payment_df, on="rental_id")

# 3. first_name ve last_name'e göre gruplama, payment.amount toplamı
result_df = merged2.groupby(["first_name", "last_name"])["amount"].sum().reset_index(name="total_spent")

# 4. Sonucu yazdır
print(result_df)

    first_name last_name  total_spent
0        AARON     SELBY       110.76
1         ADAM     GOOCH       101.78
2       ADRIAN     CLARY        74.81
3        AGNES    BISHOP        98.77
4         ALAN      KAHN       124.74
..         ...       ...          ...
594     WILLIE   MARKHAM       101.75
595      WILMA  RICHARDS        91.80
596    YOLANDA    WEAVER       110.73
597     YVONNE   WATKINS        92.79
598    ZACHARY      HITE       146.69

[599 rows x 3 columns]


### Soru 20: Her kategorideki toplam kiralama sayısını ve gelirleri bulun

In [66]:
# 1. category + film_category
merged1 = pd.merge(category_df, film_category_df, on="category_id")

# 2. + film
merged2 = pd.merge(merged1, film_df, on="film_id")

# 3. + inventory
merged3 = pd.merge(merged2, inventory_df, on="film_id",suffixes=("_film","_inv"))

# 4. + rental
merged4 = pd.merge(merged3, rental_df, on="inventory_id")

# 5. + payment
merged5 = pd.merge(merged4, payment_df, on="rental_id",suffixes=("_rent","_payment"))

# 6. Aggregation: kategoriye göre kiralama sayısı ve toplam gelir
result_df = merged5.groupby("name").agg(
    rental_count=("rental_id", "count"),
    total_revenue=("amount", "sum")
).reset_index()

# 7. Sonucu yazdır
print(result_df)

           name  rental_count  total_revenue
0        Action          1112        4375.85
1     Animation          1166        4656.30
2      Children           945        3655.55
3      Classics           939        3639.59
4        Comedy           941        4383.58
5   Documentary          1050        4217.52
6         Drama          1060        4587.39
7        Family          1096        4226.07
8       Foreign          1033        4270.67
9         Games           969        4281.33
10       Horror           846        3722.54
11        Music           830        3417.72
12          New           940        4351.62
13       Sci-Fi          1101        4756.98
14       Sports          1179        5314.21
15       Travel           837        3549.64


### Soru 21: En uzun süre kirada kalmış filmleri bulun (*)

In [75]:
# 1. film + inventory
merged1 = pd.merge(film_df, inventory_df, on="film_id")

# 2. + rental
merged2 = pd.merge(merged1, rental_df, on="inventory_id")

# 3. return_date boş olmayanları filtrele
merged2 = merged2[merged2["return_date"].notnull()]

# 4. tarih sütunlarını datetime formatına çevir
merged2["rental_date"] = pd.to_datetime(merged2["rental_date"])
merged2["return_date"] = pd.to_datetime(merged2["return_date"])

# 5. kiralama süresi (gün - float)
merged2["rental_duration"] = (merged2["return_date"] - merged2["rental_date"]).dt.total_seconds() / 86400

# 6. film başına maksimum kiralama süresi
result_df = merged2.groupby("title")["rental_duration"].max().reset_index(name="max_rental_duration")

# 7. en uzun süreli ilk 10 film
result_df = result_df.sort_values(by="max_rental_duration", ascending=False).head(10)

# 8. sonucu yazdır
print(result_df)

                  title  max_rental_duration
393     HIGHBALL POTTER             9.249306
400  HOLOCAUST HIGHBALL             9.249306
160   CONEHEADS SMOOCHY             9.248611
868        TRAMP OTHERS             9.248611
535          MASK PEACH             9.248611
625          PANIC CLUB             9.248611
602   NOTORIOUS REUNION             9.247222
38         ATTACKS HATE             9.247222
461        JERSEY SASSY             9.246528
863         TRACY CIDER             9.246528


### Soru 22: En az kiralanan 5 film hangileridir?

In [82]:

# 1. film + inventory
merged1 = pd.merge(film_df, inventory_df, on="film_id")

# 2. + rental
merged2 = pd.merge(merged1, rental_df, on="inventory_id")

# 3. film başına rental sayısı
result_df = merged2.groupby("title")["rental_id"].count().reset_index(name="rental_count")

# 4. kiralama sayısına göre artan sıralama ve ilk 5 film
result_df = result_df.sort_values(by="rental_count", ascending=True)
min_film = result_df.head(20).copy()

# index'i sütun olarak ekle
min_film['index_col'] = min_film.index

# total_rentals ve index_col'a göre sıralama yap
min_film_sorted = min_film.sort_values(by=['rental_count', 'index_col'], ascending=[True, True])

# index_col sütununu kaldır
min_film_sorted = min_film_sorted.drop(columns=['index_col'])
first_five = min_film_sorted.iloc[0:5]
print(first_five)

                title  rental_count
378    HARDLY ROBBERS             4
558       MIXED DOORS             4
866       TRAIN BUNCH             4
87   BRAVEHEART HUMAN             5
100       BUNCH MINDS             5


### Soru 23: En fazla kiralanan 5 film hangileridir?

In [27]:
# Tabloları oku
film_df = read_table_as_dataframe(conn, "film")[["film_id", "title"]]
inventory_df = read_table_as_dataframe(conn, "inventory")[["inventory_id", "film_id"]]
rental_df = read_table_as_dataframe(conn, "rental")[["rental_id", "inventory_id"]]

# Join işlemleri
merged = rental_df.merge(inventory_df, on="inventory_id").merge(film_df, on="film_id")

# Film başlığına göre kiralama sayısını hesapla
rental_counts = merged.groupby("title")["rental_id"].count().reset_index(name="rental_count")

# Kiralama sayısına göre azalan sırala ve en fazla 5 filmi seç
top5 = rental_counts.sort_values(by="rental_count", ascending=False).head(5)

print(top5)

                   title  rental_count
96    BUCKET BROTHERHOOD            34
705     ROCKETEER MOTHER            33
733        SCALAWAG DUCK            32
697  RIDGEMONT SUBMARINE            32
361       GRIT CLOCKWORK            32


### Soru 24: En fazla kazanç sağlayan 5 müşteriyi bulun

In [84]:
merged = pd.merge(customer_df, rental_df, on="customer_id")
merged = pd.merge(merged, payment_df, on="rental_id")

result_df = (
    merged.groupby(["first_name", "last_name"])
          .agg(total_spent=("amount", "sum"))
          .reset_index()
          .sort_values(by="total_spent", ascending=False)
          .head(5)
)
print(result_df)

    first_name last_name  total_spent
318       KARL      SEAL       221.55
175    ELEANOR      HUNT       216.54
105      CLARA      SHAW       195.58
389     MARION    SNYDER       194.61
474     RHONDA   KENNEDY       194.61


### Soru 25: Her filmin ortalama kiralanma süresini bulun

In [86]:
# 1. film + inventory
merged1 = pd.merge(film_df, inventory_df, on="film_id")

# 2. + rental
merged2 = pd.merge(merged1, rental_df, on="inventory_id")

# 3. return_date boş olmayanları filtrele
merged2 = merged2[merged2["return_date"].notnull()]

# 4. tarih sütunlarını datetime'a çevir
merged2["rental_date"] = pd.to_datetime(merged2["rental_date"])
merged2["return_date"] = pd.to_datetime(merged2["return_date"])

# 5. kiralama süresini gün cinsinden float olarak hesapla
merged2["rental_duration"] = (merged2["return_date"] - merged2["rental_date"]).dt.total_seconds() / 86400

# 6. film başına ortalama kiralama süresi
result_df = merged2.groupby("title")["rental_duration"].mean().reset_index(name="avg_rental_duration")

# 7. sonucu yazdır
print(result_df)


                 title  avg_rental_duration
0     ACADEMY DINOSAUR             4.997128
1       ACE GOLDFINGER             5.642708
2     ADAPTATION HOLES             3.465625
3     AFFAIR PREJUDICE             4.758333
4          AFRICAN EGG             7.106629
..                 ...                  ...
953     YOUNG LANGUAGE             4.624405
954         YOUTH KICK             5.562963
955       ZHIVAGO CORE             5.950087
956  ZOOLANDER FICTION             5.542647
957          ZORRO ARK             4.539180

[958 rows x 2 columns]


### Soru 26: Her türde en popüler filmi bulun

In [88]:
# 1. category + film_category
merged1 = pd.merge(category_df, film_category_df, on="category_id")

# 2. + film
merged2 = pd.merge(merged1, film_df, on="film_id")

# 3. + inventory
merged3 = pd.merge(merged2, inventory_df, on="film_id",suffixes=("_film","-inv"))

# 4. + rental
merged4 = pd.merge(merged3, rental_df, on="inventory_id")

# 5. 
result_df = merged4.groupby(["name", "title"]).agg(
    rental_count=pd.NamedAgg(column="rental_id", aggfunc="count")
).reset_index()

# 6. rental_count'a göre azalan sıralama ve ilk satırı al
result_df = result_df.sort_values(by="rental_count", ascending=False).head(1)

# 7. kolon ismini alias gibi değiştirme
result_df = result_df.rename(columns={"name": "genre"})

print(result_df)


      genre               title  rental_count
909  Travel  BUCKET BROTHERHOOD            34


### Soru 27: Her türde en fazla gelir sağlayan filmi bulun

In [90]:
# 1. category + film_category
merged1 = pd.merge(category_df, film_category_df, on="category_id")

# 2. + film
merged2 = pd.merge(merged1, film_df, on="film_id",suffixes=("_cate","_film"))

# 3. + inventory
merged3 = pd.merge(merged2, inventory_df, on="film_id",suffixes=("_merg","_inv"))

# 4. + rental
merged4 = pd.merge(merged3, rental_df, on="inventory_id",suffixes=("_merg3","_rent"))

# 5. + payment
merged5 = pd.merge(merged4, payment_df, on="rental_id",suffixes=("_merg4","_payment"))

result_df = (
    merged5.groupby(["name", "title"])
    .agg(total_revenue=("amount", "sum"))
    .sort_values(by="total_revenue", ascending=False)
    .head(1)
    .reset_index()
)
print(result_df)

    name             title  total_revenue
0  Music  TELEGRAPH VOYAGE         231.73


### Soru 28: En çok DVD iade etmeyen müşteriyi bulun

In [92]:
# 1. customer + rental
merged = pd.merge(customer_df, rental_df, on="customer_id")

# 2. Henüz iade edilmemişleri filtrele
not_returned = merged[merged["return_date"].isnull()]

# 3. Ad ve soyada göre grupla, iade edilmeyen kiralamaları say
grouped = (
    not_returned.groupby(["first_name", "last_name"])
    .agg(unreturned_count=pd.NamedAgg(column="rental_id", aggfunc="count"))
    .reset_index()
)

# 4. En çok iade edilmeyen kiralamaya sahip müşteriyi al
result_df = grouped.sort_values(by="unreturned_count", ascending=False).head(1)

# 5. Sonucu yazdır
print(result_df)


    first_name last_name  unreturned_count
144      TAMMY   SANDERS                 3


### Soru 29: En fazla kiralama yapan 5 çalışanı bulun

In [94]:
# 1. staff + rental
merged = pd.merge(staff_df, rental_df, on="staff_id")

# 2. Ad ve soyad bilgisine göre grupla, rental_id'yi say
result_df = (
    merged.groupby(["first_name", "last_name"])
    .agg(rental_count=pd.NamedAgg(column="rental_id", aggfunc="count"))
    .reset_index()
    .sort_values(by="rental_count", ascending=False)
    .head(5)
)

print(result_df)

  first_name last_name  rental_count
1       Mike   Hillyer          8040
0        Jon  Stephens          8004


### Soru 30: En fazla kiralama yapan 5 müşteri hangi şubeden kiralama yapmış?(*)

In [111]:
# 1. customer + rental
merged1 = pd.merge(customer_df, rental_df, on="customer_id")

# 2. + inventory (store_id burada var)
merged2 = pd.merge(merged1, inventory_df[['inventory_id', 'store_id']], on="inventory_id")

# 3. store_id, first_name, last_name bazında gruplama ve kiralama sayısı
result_df = (
    merged2
    .groupby(['store_id_x', 'first_name', 'last_name'])
    .agg(rental_count=('rental_id', 'count'))
    .reset_index()
)

# 4. rental_count'e göre azalan sırala ve ilk 5 kişiyi seç
result_df = result_df.sort_values(by='rental_count', ascending=False).head(5)

print(result_df)


     store_id_x first_name last_name  rental_count
95            1    ELEANOR      HUNT            46
477           2       KARL      SEAL            45
54            1      CLARA      SHAW            42
204           1     MARCIA      DEAN            42
572           2      TAMMY   SANDERS            41


### Soru 31: Her türde en az kiralanan filmi bulun

In [113]:
# 1. category + film_category
merged1 = pd.merge(category_df, film_category_df, on="category_id")

# 2. + film
merged2 = pd.merge(merged1, film_df, on="film_id")

# 3. + inventory
merged3 = pd.merge(merged2, inventory_df, on="film_id", suffixes=("_left", "_right"))

# 4. + rental
merged4 = pd.merge(merged3, rental_df, on="inventory_id")

# 5. Grup ve kiralama sayısı
result_df = (
    merged4.groupby(["name", "title"])
    .agg(rental_count=pd.NamedAgg(column="rental_id", aggfunc="count"))
    .reset_index()
)

# 6. Çoklu sıralama: önce kiralama sayısı (azdan çoğa)
result_df = result_df.sort_values(by=["rental_count", "name", "title"], ascending=[True, True, True]).head(1)

# 7. Kolon adını düzelt
result_df = result_df.rename(columns={"name": "genre"})

# 8. Sonucu yazdır
print(result_df)

           genre           title  rental_count
315  Documentary  HARDLY ROBBERS             4


### Soru 32: En çok kiralama yapan 5 müşteri hangi şehirde?

In [115]:
# 1. customer + address (customer_df, address_df)
merged1 = pd.merge(customer_df, address_df, on="address_id")

# 2. + city (address_df, city_df)
merged2 = pd.merge(merged1, city_df, on="city_id")

# 3. + rental (customer ve rental ilişkisi)
merged3 = pd.merge(merged2, rental_df, on="customer_id",suffixes=("_left","_right"))

# 4. Gruplama ve kiralama sayısı
result_df = (
    merged3.groupby(["city", "first_name", "last_name"])
    .agg(rental_count=pd.NamedAgg(column="rental_id", aggfunc="count"))
    .reset_index()
    .sort_values(by="rental_count", ascending=False)
    .head(5)
)

print(result_df)

            city first_name last_name  rental_count
433  Saint-Denis    ELEANOR      HUNT            46
96    Cape Coral       KARL      SEAL            45
333    Molodetno      CLARA      SHAW            42
518        Tanza     MARCIA      DEAN            42
103     Changhwa      TAMMY   SANDERS            41


### Soru 33: En çok kazanç sağlayan 5 müşteriyi hangi şehirde bulun?

In [117]:
# 1. customer + address
merged1 = pd.merge(customer_df, address_df, on="address_id")

# 2. + city
merged2 = pd.merge(merged1, city_df, on="city_id")

# 3. + rental
merged3 = pd.merge(merged2, rental_df, on="customer_id",suffixes=("_left","_right"))

# 4. + payment
merged4 = pd.merge(merged3, payment_df, on="rental_id")

# 5. Grup ve toplam ödeme
result_df = (
    merged4.groupby(["city", "first_name", "last_name"])
    .agg(total_spent=pd.NamedAgg(column="amount", aggfunc="sum"))
    .reset_index()
    .sort_values(by="total_spent", ascending=False)
    .head(5)
)

print(result_df)

                    city first_name last_name  total_spent
96            Cape Coral       KARL      SEAL       221.55
433          Saint-Denis    ELEANOR      HUNT       216.54
333            Molodetno      CLARA      SHAW       195.58
23             Apeldoorn     RHONDA   KENNEDY       194.61
447  Santa Brbara dOeste     MARION    SNYDER       194.61


### Soru 34: En çok kiralanan 5 filmi hangi şehirde bulun?

In [120]:
# 1. city + address (city_id)
merged1 = pd.merge(city_df, address_df, on="city_id")

# 2. + customer (address_id)
merged2 = pd.merge(merged1, customer_df, on="address_id")

# 3. + rental (customer_id)
merged3 = pd.merge(merged2, rental_df, on="customer_id")

# 4. + inventory (inventory_id)
merged4 = pd.merge(merged3, inventory_df[['inventory_id', 'film_id']], on="inventory_id")

# 5. + film (film_id)
merged5 = pd.merge(merged4, film_df[['film_id', 'title']], on="film_id")

# 6. Grup ve sayım
result_df = (
    merged5
    .groupby(["city", "title"])
    .agg(rental_count=("rental_id", "count"))
    .reset_index()
)

# 7. Kiralama sayısına göre azalan sırala ve ilk 5'i al
result_df = result_df.sort_values(by="rental_count", ascending=False).head(5)
print(result_df)

                        city              title  rental_count
7828                    Lima    DISCIPLE MOTHER             3
7308                  Kurgan   DETECTIVE VISION             3
12735               Sorocaba    CADDYSHACK JEDI             3
14182                Trshavn  FLATLINERS KILLER             3
3793   Donostia-San Sebastin    REIGN GENTLEMEN             2


### Soru 35: En az kiralanan 5 filmi hangi şehirde bulun? (*)

In [24]:
# Tüm gerekli tabloları birleştirelim
merged_df = pd.merge(city_df, address_df, on='city_id')
merged_df = pd.merge(merged_df, customer_df, on='address_id')
merged_df = pd.merge(merged_df, rental_df, on='customer_id',suffixes=("_left","_right"))
merged_df = pd.merge(merged_df, inventory_df, on='inventory_id')
merged_df = pd.merge(merged_df, film_df, on='film_id',suffixes=("_merged","_film"))

# Şehir ve film başına kiralama sayısını hesaplayalım
result_df = merged_df.groupby(['city', 'title']).agg(
    rental_count=('rental_id', 'count')
).reset_index()

# En az kiralanan 5 filmi bulalım
result_df = result_df.sort_values('rental_count').head(5)

# Sonucu göstermek için sütunları yeniden düzenleyelim
result_df = result_df[['city', 'title', 'rental_count']]

print(result_df)

                     city                title  rental_count
15826     s-Hertogenbosch         WORST BANGER             1
17     A Corua (La Corua)      JEEPERS WEDDING             1
18     A Corua (La Corua)  LADYBUGS ARMAGEDDON             1
19     A Corua (La Corua)     LOVERBOY ATTACKS             1
20     A Corua (La Corua)       MASSACRE USUAL             1


### Soru 36: En çok kazanç sağlayan 5 filmi hangi şehirde bulun?

In [125]:
# 1. city + address
merged1 = pd.merge(city_df, address_df, on="city_id")

# 2. + customer
merged2 = pd.merge(merged1, customer_df, on="address_id")

# 3. + rental
merged3 = pd.merge(merged2, rental_df, on="customer_id")

# 4. + inventory (sadece gerekli sütunlar alınır, last_update çıkarılır)
merged4 = pd.merge(merged3, inventory_df[['inventory_id', 'film_id']], on="inventory_id")

# 5. + film (sadece gerekli sütunlar alınır)
merged5 = pd.merge(merged4, film_df[['film_id', 'title']], on="film_id")

# 6. + payment
merged6 = pd.merge(merged5, payment_df[['rental_id', 'amount']], on="rental_id")

# 7. Şehir ve film bazında toplam geliri hesapla
result_df = (
    merged6
    .groupby(["city", "title"])
    .agg(total_revenue=pd.NamedAgg(column="amount", aggfunc="sum"))
    .reset_index()
    .sort_values(by="total_revenue", ascending=False)
    .head(5)
)

print(result_df)

                      city             title  total_revenue
10565                Plock  CALIFORNIA BIRDS          18.98
10752          Probolinggo      MINDS TRUMAN          18.98
6650                Kamyin    ROSES TREASURE          17.98
11854  Santa Brbara dOeste         WIFE TURN          17.98
3520                Datong      EAGLES PANKY          16.98


### Soru 37: En az kazanç sağlayan 5 filmi hangi şehirde bulun?

In [128]:
# 1. city + address
merged1 = pd.merge(city_df[['city_id', 'city']], address_df[['address_id', 'city_id']], on="city_id")

# 2. + customer
merged2 = pd.merge(merged1, customer_df[['customer_id', 'address_id']], on="address_id")

# 3. + rental
merged3 = pd.merge(merged2, rental_df[['rental_id', 'customer_id', 'inventory_id']], on="customer_id")

# 4. + inventory (sadece gerekli sütunlar: film_id)
merged4 = pd.merge(merged3, inventory_df[['inventory_id', 'film_id']], on="inventory_id")

# 5. + film (sadece film adı gerekli)
merged5 = pd.merge(merged4, film_df[['film_id', 'title']], on="film_id")

# 6. + payment (amount sütunu gerekli)
merged6 = pd.merge(merged5, payment_df[['rental_id', 'amount']], on="rental_id")

# 7. Şehir ve film bazında toplam geliri hesapla
result_df = (
    merged6.groupby(["city", "title"])
    .agg(total_revenue=pd.NamedAgg(column="amount", aggfunc="sum"))
    .reset_index()
    .sort_values(by="total_revenue", ascending=True)
    .head(30)
)

# Sıralama için eski index ekle ve eşitlik durumunda index'e göre sırala
result_df["old_index"] = result_df.index
result_df = result_df.sort_values(by=["total_revenue", "old_index"], ascending=[True, True]).head(5)

result_df = result_df.drop(columns=["old_index"]).reset_index(drop=True)
print(result_df)

                    city              title  total_revenue
0  Allappuzha (Alleppey)     DEER VIRGINIAN            0.0
1    Aparecida de Goinia       WOMEN DORADO            0.0
2             Avellaneda  MOVIE SHAKESPEARE            0.0
3              Balurghat  CURTAIN VIDEOTAPE            0.0
4             Battambang      MINORITY KISS            0.0


### Soru 38: En fazla kiralama yapan müşteri hangi filmleri kiralamış?

In [71]:
# 1. Kiralama sayısına göre müşteri bazında kiralama sayısı hesapla
rental_counts = rental_df.groupby("customer_id").size()

# 2. En çok kiralama yapan müşteri_id
top_customer_id = rental_counts.idxmax()

# 3. customer_df'den bu müşterinin bilgileri
top_customer = customer_df[customer_df["customer_id"] == top_customer_id]

# 4. rental_df'den sadece bu müşterinin kiralamaları
top_customer_rentals = rental_df[rental_df["customer_id"] == top_customer_id]

# 5. rental + inventory + film + film_category + category tablolarını sırayla birleştir
merged = top_customer_rentals.merge(inventory_df, on="inventory_id") \
    .merge(film_df, on="film_id") \
    .merge(film_category_df, on="film_id",suffixes=("_left","_right")) \
    .merge(category_df, on="category_id")

# 6. genre (category.name) bazında kiralama sayısını hesapla
result_df = merged.groupby("name").agg(rental_count=("rental_id", "count")).reset_index()

# 7. Müşteri bilgilerini (isim, soyisim) ekle
result_df["first_name"] = top_customer.iloc[0]["first_name"]
result_df["last_name"] = top_customer.iloc[0]["last_name"]

# 8. Kolon sırasını düzenle
result_df = result_df[["first_name", "last_name", "name", "rental_count"]]

# 9. Sonucu yazdır
print(result_df)


   first_name last_name         name  rental_count
0     ELEANOR      HUNT       Action             3
1     ELEANOR      HUNT     Classics             4
2     ELEANOR      HUNT       Comedy             2
3     ELEANOR      HUNT  Documentary             3
4     ELEANOR      HUNT        Drama             1
5     ELEANOR      HUNT       Family             6
6     ELEANOR      HUNT      Foreign             4
7     ELEANOR      HUNT        Games             3
8     ELEANOR      HUNT       Horror             4
9     ELEANOR      HUNT        Music             1
10    ELEANOR      HUNT          New             2
11    ELEANOR      HUNT       Sci-Fi             7
12    ELEANOR      HUNT       Sports             1
13    ELEANOR      HUNT       Travel             5


### Soru 39: En az kiralama yapan müşteri hangi filmleri kiralamış?

In [30]:
# Verilen read_table_as_dataframe fonksiyonunu kullanarak tabloları yükle
customer_df = read_table_as_dataframe(conn, "customer")[["customer_id", "first_name", "last_name"]]
rental_df = read_table_as_dataframe(conn, "rental")[["rental_id", "inventory_id", "customer_id"]]
inventory_df = read_table_as_dataframe(conn, "inventory")[["inventory_id", "film_id"]]
film_df = read_table_as_dataframe(conn, "film")[["film_id"]]
film_category_df = read_table_as_dataframe(conn, "film_category")[["film_id", "category_id"]]
category_df = read_table_as_dataframe(conn, "category")[["category_id", "name"]]

# 1) Her müşterinin kiralama sayısını hesapla
rental_counts = rental_df.groupby("customer_id")["rental_id"].count().reset_index(name="rental_count")

# 2) En az kiralama yapan müşterinin customer_id'sini bul
min_rental_customer_id = rental_counts.sort_values("rental_count").iloc[0]["customer_id"]

# 3) En az kiralama yapan müşterinin kiralamalarını filtrele
cust_rentals = rental_df[rental_df["customer_id"] == min_rental_customer_id]

# 4) Tabloları birleştir
merged = (
    cust_rentals
    .merge(inventory_df, on="inventory_id")
    .merge(film_df, on="film_id")
    .merge(film_category_df, on="film_id")
    .merge(category_df, on="category_id")
    .merge(customer_df, on="customer_id")
)

# 5) Türlere göre gruplama ve kiralama sayısı
result = (
    merged.groupby(["first_name", "last_name", "name"])
    .agg(rental_count=("rental_id", "count"))
    .reset_index()
    .rename(columns={"name": "genre"})
)

print(result)

  first_name last_name        genre  rental_count
0      BRIAN     WYMAN       Action             3
1      BRIAN     WYMAN       Comedy             1
2      BRIAN     WYMAN  Documentary             3
3      BRIAN     WYMAN        Drama             2
4      BRIAN     WYMAN       Horror             1
5      BRIAN     WYMAN          New             2


### Soru 40: En çok kazanç sağlayan müşteri hangi filmleri kiralamış?

In [130]:
rental_payment_df = rental_df.merge(payment_df, on="rental_id")
rental_payment_customer_df = rental_payment_df.merge(customer_df,left_on="customer_id_y",right_on="customer_id")

# En çok ödeme yapan müşteriyi bul
customer_revenue_df = rental_payment_customer_df.groupby("customer_id")["amount"].sum().reset_index()
top_customer_id = customer_revenue_df.sort_values("amount", ascending=False).iloc[0]["customer_id"]

# O müşterinin kiralama kayıtları
top_rentals_df = rental_df[rental_df["customer_id"] == top_customer_id]
top_inventory_df = top_rentals_df.merge(inventory_df, on="inventory_id")
top_films_df = top_inventory_df.merge(film_df, on="film_id")
top_full_df = top_films_df.merge(payment_df, on="rental_id", suffixes=('', '_payment'))
top_full_df = top_full_df.merge(customer_df[["customer_id", "first_name", "last_name"]], on="customer_id")

# Film bazında gelir
result_df = top_full_df.groupby(["first_name", "last_name", "title"])["amount"].sum().reset_index()
result_df = result_df.rename(columns={"amount": "total_revenue"})

print(result_df)

   first_name last_name                  title  total_revenue
0        KARL      SEAL          BETRAYED REAR           4.99
1        KARL      SEAL       BIKINI BORROWERS           6.99
2        KARL      SEAL         BLUES INSTINCT           2.99
3        KARL      SEAL    BRINGING HYSTERICAL           2.99
4        KARL      SEAL         CYCLONE FAMILY           4.99
5        KARL      SEAL             DATE SPEED           3.99
6        KARL      SEAL           DESIRE ALIEN           2.99
7        KARL      SEAL       DESTINY SATURDAY           4.99
8        KARL      SEAL             DUMBO LUST           0.99
9        KARL      SEAL           DURHAM PANKY           4.99
10       KARL      SEAL       ENGLISH BULWORTH           5.99
11       KARL      SEAL         FIDELITY DEVIL           4.99
12       KARL      SEAL       FOOL MOCKINGBIRD          10.99
13       KARL      SEAL  FORRESTER COMANCHEROS           4.99
14       KARL      SEAL            GANGS PRIDE           2.99
15      

### Soru 41: En az kazanç sağlayan müşteri hangi filmleri kiralamış?

In [132]:
# 1. rental + payment → ödemeleri rental üzerinden birleştir
rental_payment_df = rental_df.merge(payment_df, on="rental_id")

# 2. En az ödeme yapan müşteriyi bul
# Muhtemelen payment tablosundan gelen customer_id_y kullanılmalı
customer_revenue_df = rental_payment_df.groupby("customer_id_y")["amount"].sum().reset_index()
customer_revenue_df = customer_revenue_df.rename(columns={"customer_id_y": "customer_id"})
least_paying_customer_id = customer_revenue_df.sort_values("amount", ascending=True).iloc[0]["customer_id"]

# 3. Bu müşterinin rental kayıtlarını al
least_customer_rentals = rental_df[rental_df["customer_id"] == least_paying_customer_id]

# 4. rental → inventory → film (filmleri öğren)
inventory_joined = least_customer_rentals.merge(inventory_df, on="inventory_id")
films_joined = inventory_joined.merge(film_df, on="film_id")

# 5. Bu kiralamalara karşılık gelen ödemeleri al
films_with_payment = films_joined.merge(payment_df[['rental_id', 'amount']], on="rental_id")

# 6. Müşteri bilgilerini ekle
films_with_customer = films_with_payment.merge(
    customer_df[["customer_id", "first_name", "last_name"]],
    on="customer_id"
)

# 7. Film bazında ödeme toplamı
result_df = films_with_customer.groupby(["first_name", "last_name", "title"])["amount"].sum().reset_index()

# 8. Sütun adlarını SQL'deki gibi 
result_df = result_df.rename(columns={"amount": "total_revenue"})

print(result_df)

   first_name last_name                   title  total_revenue
0    CAROLINE    BOWMAN         ARMAGEDDON LOST           0.99
1    CAROLINE    BOWMAN        CAMELOT VACATION           0.99
2    CAROLINE    BOWMAN         DISCIPLE MOTHER           5.99
3    CAROLINE    BOWMAN        EMPIRE MALKOVICH           0.99
4    CAROLINE    BOWMAN          FRENCH HOLIDAY           7.99
5    CAROLINE    BOWMAN       GOODFELLAS SALUTE           4.99
6    CAROLINE    BOWMAN       HOOSIERS BIRDCAGE           4.99
7    CAROLINE    BOWMAN         ILLUSION AMELIE           0.99
8    CAROLINE    BOWMAN    IMPOSSIBLE PREJUDICE           5.99
9    CAROLINE    BOWMAN              JADE BUNCH           2.99
10   CAROLINE    BOWMAN         KILLER INNOCENT           2.99
11   CAROLINE    BOWMAN              MASK PEACH           3.99
12   CAROLINE    BOWMAN  RESURRECTION SILVERADO           0.99
13   CAROLINE    BOWMAN             TEXAS WATCH           0.99
14   CAROLINE    BOWMAN         WHISPERER GIANT        

### Soru 42: En az kazanç sağlayan müşteri hangi türde en fazla film kiralamış?

In [135]:
# 1. rental ile payment birleştir
rental_payment_df = rental_df.merge(payment_df, on='rental_id')

# 2. müşteri bazında toplam ödeme
customer_payment_sum = rental_payment_df.groupby('customer_id_x' if 'customer_id_x' in rental_payment_df.columns else 'customer_id')['amount'].sum()

# 3. en az ödeme yapan müşteri id'si
least_paying_customer_id = customer_payment_sum.idxmin()

# 4. o müşterinin kiralamalarını al
customer_rentals = rental_df[rental_df['customer_id'] == least_paying_customer_id]

# 5. kiralamaları inventory ile birleştir
rental_inventory = customer_rentals.merge(inventory_df, on='inventory_id')

# 6. inventory → film
rental_film = rental_inventory.merge(film_df, on='film_id')

# 7. film → film_category
rental_film_category = rental_film.merge(film_category_df, on='film_id',suffixes=('_film', '_film_cat'))

# 8. film_category → category
rental_film_category = rental_film_category.merge(category_df[['category_id', 'name']], on='category_id')

# 9. genre bazında kiralama sayısı
genre_counts = rental_film_category.groupby('name').size().reset_index(name='total_count')

# 10. müşteri bilgilerini al
customer_info = customer_df[customer_df['customer_id'] == least_paying_customer_id][['first_name', 'last_name']].iloc[0]

# 11. müşteri bilgilerini genre_counts tablosuna ekle
genre_counts['first_name'] = customer_info['first_name']
genre_counts['last_name'] = customer_info['last_name']

# 12. kolon sırasını SQL sorgusundaki gibi ayarla
genre_counts = genre_counts[['first_name', 'last_name', 'name', 'total_count']]

# 13. kolon ismini genre olarak değiştir
genre_counts = genre_counts.rename(columns={'name': 'genre'})

print(genre_counts)

  first_name last_name        genre  total_count
0   CAROLINE    BOWMAN     Children            1
1   CAROLINE    BOWMAN  Documentary            1
2   CAROLINE    BOWMAN       Family            2
3   CAROLINE    BOWMAN      Foreign            3
4   CAROLINE    BOWMAN       Horror            1
5   CAROLINE    BOWMAN          New            1
6   CAROLINE    BOWMAN       Sci-Fi            4
7   CAROLINE    BOWMAN       Sports            1
8   CAROLINE    BOWMAN       Travel            1


### Soru 43: En çok kiralanan film hangi çalışan tarafından kiralanmış?

In [137]:
# 1. rental ve inventory'yi birleştir (rental'in film bilgisini almak için)
rental_inventory = rental_df.merge(inventory_df, on='inventory_id')

# 2. film bilgilerini ekle
rental_inventory_film = rental_inventory.merge(film_df, on='film_id')

# 3. film bazında kiralama sayısını hesapla
film_rental_counts = rental_inventory_film.groupby('film_id').size().reset_index(name='rental_count')

# 4. en çok kiralanan film_id'sini bul
most_rented_film_id = film_rental_counts.sort_values('rental_count', ascending=False).iloc[0]['film_id']

# 5. en çok kiralanan filmle ilgili rental + inventory + film kaydını filtrele
most_rented_film_rentals = rental_inventory_film[rental_inventory_film['film_id'] == most_rented_film_id]

# 6. staff bilgilerini eklemek için staff_df ile birleştir
result = most_rented_film_rentals.merge(staff_df, on='staff_id')

# 7. personel bazında toplam kiralama sayısını hesapla
staff_rental_counts = result.groupby(['first_name', 'last_name', 'title']).size().reset_index(name='rental_count')

print(staff_rental_counts)

  first_name last_name               title  rental_count
0        Jon  Stephens  BUCKET BROTHERHOOD            18
1       Mike   Hillyer  BUCKET BROTHERHOOD            16


### Soru 44: En az kiralanan film hangi çalışan tarafından kiralanmış?

In [139]:
# 1. Film kiralama sayılarını hesapla
film_rental_counts = (
    rental_df
    .merge(inventory_df, on="inventory_id")
    .groupby("film_id")
    .size()
    .reset_index(name="rental_count")
)

# 2. En az kiralanan filmi bul
min_rental_count = film_rental_counts["rental_count"].min()
least_rented_film_id = film_rental_counts.loc[film_rental_counts["rental_count"] == min_rental_count, "film_id"].values[0]

# 3. En az kiralanan filmle ilgili tüm rental verilerini al
least_rented_film_rentals = (
    rental_df
    .merge(inventory_df, on="inventory_id")
    .query("film_id == @least_rented_film_id")
)
# Gereksiz sütunları kaldır
film_df_clean = film_df.drop(columns=["last_update"])
staff_df_clean = staff_df.drop(columns=["last_update", "address_id", "picture", "email"], errors="ignore")

# 4. Rental, Staff ve Film tablolarını birleştir
result_df = (
    least_rented_film_rentals
    .merge(staff_df_clean, on="staff_id")
    .merge(film_df_clean, on="film_id")
)

# 5. Personel ve film adına göre grupla ve say
final_df = (
    result_df
    .groupby(["first_name", "last_name", "title"])
    .size()
    .reset_index(name="rental_count")
)

print(final_df)

  first_name last_name           title  rental_count
0        Jon  Stephens  HARDLY ROBBERS             3
1       Mike   Hillyer  HARDLY ROBBERS             1


### Soru 45: En çok kazanç sağlayan film hangi çalışan tarafından kiralanmış?

In [141]:
# 1. payment, rental, inventory ve film tablolarını birleştir
payment_rental_film_df = (
    payment_df
    .merge(rental_df, on="rental_id")
    .merge(inventory_df, on="inventory_id")
    .merge(film_df[["film_id", "title"]], on="film_id")
)

# 2. film_id bazlı toplam geliri hesapla
film_revenue_df = (
    payment_rental_film_df
    .groupby("film_id")["amount"]
    .sum()
    .reset_index(name="total_revenue")
)

# 3. En yüksek gelir getiren film_id'yi bul
max_revenue_film_id = (
    film_revenue_df
    .sort_values("total_revenue", ascending=False)
    .iloc[0]["film_id"]
)

# 4. En çok gelir getiren filme ait tüm işlemleri filtrele
max_film_df = payment_rental_film_df.query("film_id == @max_revenue_film_id")

# 5. staff bilgilerini rental_df üzerinden staff_id ile bağla
staff_info_df = (
    rental_df[["rental_id", "staff_id"]]
    .merge(staff_df[["staff_id", "first_name", "last_name"]], on="staff_id")
)

# 6. max_film_df ile personel bilgilerini birleştir
max_film_with_staff = (
    max_film_df
    .merge(staff_info_df, on="rental_id")
)

# 7. Gruplama ve toplam gelir hesaplama
final_df = (
    max_film_with_staff
    .groupby(["first_name", "last_name", "title"])["amount"]
    .sum()
    .reset_index(name="total_revenue")
)

print(final_df)

  first_name last_name             title  total_revenue
0        Jon  Stephens  TELEGRAPH VOYAGE         147.83
1       Mike   Hillyer  TELEGRAPH VOYAGE          83.90


### Soru 46: En az kazanç sağlayan film hangi çalışan tarafından kiralanmış?(*)

In [144]:
# 1. rental + payment
rental_payment_df = pd.merge(rental_df, payment_df, on="rental_id")

# 2. inventory ile birleştir
rental_payment_inventory_df = pd.merge(rental_payment_df, inventory_df[['inventory_id', 'film_id']], on="inventory_id")

# 3. film ile birleştir
film_payment_df = pd.merge(rental_payment_inventory_df, film_df[['film_id', 'title']], on="film_id")

# 4. film bazında toplam gelir
film_revenue = film_payment_df.groupby(['film_id', 'title'])['amount'].sum().reset_index(name="total_revenue")

# 5. en az gelire sahip film_id'yi bul
least_revenue_film_id = film_revenue.sort_values(by="total_revenue").iloc[0]['film_id']

# 6. rental + inventory + film + staff + payment birleştir
merged1 = pd.merge(rental_df, inventory_df[['inventory_id', 'film_id']], on="inventory_id")
merged2 = pd.merge(merged1, film_df[['film_id', 'title']], on="film_id")
merged3 = pd.merge(merged2, staff_df[['staff_id', 'first_name', 'last_name']], on="staff_id")
merged4 = pd.merge(merged3, payment_df[['rental_id', 'amount']], on="rental_id")

# 7. sadece en az gelire sahip filmi filtrele
filtered = merged4[merged4['film_id'] == least_revenue_film_id]

# 8. staff ve film bazında toplam gelir
result_df = (
    filtered.groupby(['first_name', 'last_name', 'title'])['amount']
    .sum()
    .reset_index(name="total_revenue")
)
print(result_df)


  first_name last_name        title  total_revenue
0        Jon  Stephens  TEXAS WATCH           2.97
1       Mike   Hillyer  TEXAS WATCH           2.97


### Soru 47: En çok kiralanan film hangi mağazada kiralanmış?

In [148]:
# 1. rental + inventory + film birleşimi → film bazında kaç kez kiralandığını hesapla
film_rental_count_df = (
    rental_df
    .merge(inventory_df, on="inventory_id")
    .merge(film_df[["film_id", "title"]], on="film_id")
)

# 2. film_id bazında kiralama sayısı
film_rental_counts = (
    film_rental_count_df
    .groupby(["film_id", "title"])
    .size()
    .reset_index(name="rental_count")
)

# 3. En çok kiralanan film_id'yi bul
most_rented_film_row = film_rental_counts.sort_values("rental_count", ascending=False).iloc[0]
most_rented_film_id = most_rented_film_row["film_id"]
most_rented_film_title = most_rented_film_row["title"]

# 4. Bu film_id'ye ait tüm rental verisini al
most_rented_film_df = film_rental_count_df.query("film_id == @most_rented_film_id")

# 5. staff_id üzerinden store_id'yi alabilmek için rental + staff + store ilişkisini kur
rental_staff_store_df = (
    rental_df[["rental_id", "staff_id"]]
    .merge(staff_df[["staff_id", "store_id"]], on="staff_id")
)

# 6. film verisi ile mağaza bilgilerini bağla
merged_df = (
    most_rented_film_df[["rental_id", "title"]]  # rental_id gerekli
    .merge(rental_staff_store_df, on="rental_id")
)

# 7. store_id ve film başlığına göre grupla → say
final_df = (
    merged_df
    .groupby(["store_id", "title"])
    .size()
    .reset_index(name="rental_count")
)

print(final_df)

   store_id               title  rental_count
0         1  BUCKET BROTHERHOOD            16
1         2  BUCKET BROTHERHOOD            18


### Soru 48: En az kiralanan film hangi mağazada kiralanmış?

In [150]:
# 1. rental + inventory + film birleşimi
film_rental_df = (
    rental_df
    .merge(inventory_df, on="inventory_id")
    .merge(film_df[["film_id", "title"]], on="film_id")
)

# 2. film bazında kiralama sayısını hesapla
film_rental_counts = (
    film_rental_df
    .groupby(["film_id", "title"])
    .size()
    .reset_index(name="rental_count")
)

# 3. En az kiralanan film_id'yi al
least_rented_film_row = film_rental_counts.sort_values("rental_count", ascending=True).iloc[0]
least_rented_film_id = least_rented_film_row["film_id"]
least_rented_film_title = least_rented_film_row["title"]

# 4. Bu film_id'ye ait kiralama kayıtlarını al
least_rented_film_df = film_rental_df.query("film_id == @least_rented_film_id")

# 5. store_id’ye ulaşmak için: rental → staff → store
rental_staff_store_df = (
    rental_df[["rental_id", "staff_id"]]
    .merge(staff_df[["staff_id", "store_id"]], on="staff_id")
)

# 6. Film ile store_id'yi birleştir
merged_df = (
    least_rented_film_df[["rental_id", "title"]]
    .merge(rental_staff_store_df, on="rental_id")
)

# 7. store_id ve title’a göre grupla ve say
final_df = (
    merged_df
    .groupby(["store_id", "title"])
    .size()
    .reset_index(name="rental_count")
)

print(final_df)

   store_id        title  rental_count
0         2  TRAIN BUNCH             4


### Soru 49: En çok kazanç sağlayan film hangi mağazada kiralanmış?

In [152]:
# 1. payment + rental + inventory + film birleşimi
payment_rental_film_df = (
    payment_df
    .merge(rental_df, on="rental_id")
    .merge(inventory_df, on="inventory_id")
    .merge(film_df[["film_id", "title"]], on="film_id")
)

# 2. film_id bazında toplam gelir
film_revenue_df = (
    payment_rental_film_df
    .groupby(["film_id", "title"])["amount"]
    .sum()
    .reset_index(name="total_revenue")
)

# 3. En çok gelir getiren film_id
most_revenue_film_row = film_revenue_df.sort_values("total_revenue", ascending=False).iloc[0]
most_revenue_film_id = most_revenue_film_row["film_id"]
most_revenue_film_title = most_revenue_film_row["title"]

# 4. Bu filme ait ödeme-kiralama kayıtlarını al
most_revenue_film_df = payment_rental_film_df.query("film_id == @most_revenue_film_id")

# 5. rental_id üzerinden staff ve store bilgisi ekle
rental_staff_store_df = (
    rental_df[["rental_id", "staff_id"]]
    .merge(staff_df[["staff_id", "store_id"]], on="staff_id")
)

# 6. mağaza bilgileri ile filmi birleştir
merged_df = (
    most_revenue_film_df[["rental_id", "title"]]
    .merge(rental_staff_store_df, on="rental_id")
)

# 7. store_id ve title’a göre grupla ve say
final_df = (
    merged_df
    .groupby(["store_id", "title"])
    .size()
    .reset_index(name="rental_count")
)

print(final_df)

   store_id             title  rental_count
0         1  TELEGRAPH VOYAGE            10
1         2  TELEGRAPH VOYAGE            17


### Soru 50: En az kazanç sağlayan film hangi mağazada kiralanmış?(*)

In [156]:
# 1. payment + rental + inventory + film birleşimi
payment_rental_film_df = (
    payment_df
    .merge(rental_df, on="rental_id")
    .merge(inventory_df, on="inventory_id")
    .merge(film_df[["film_id", "title"]], on="film_id")
)

# 2. film_id ve title bazında toplam geliri hesapla
film_revenue_df = (
    payment_rental_film_df
    .groupby(["film_id", "title"])["amount"]
    .sum()
    .reset_index(name="total_revenue")
)

# 3. En az gelir getiren film_id ve başlığını al
least_revenue_row = film_revenue_df.sort_values("total_revenue", ascending=True).iloc[0]
least_revenue_film_id = least_revenue_row["film_id"]
least_revenue_film_title = least_revenue_row["title"]

# 4. Bu filme ait rental ve payment kayıtlarını filtrele
least_revenue_film_df = payment_rental_film_df.query("film_id == @least_revenue_film_id")

# 5. rental_id üzerinden staff ve store bilgilerini birleştir
rental_staff_store_df = (
    rental_df[["rental_id", "staff_id"]]
    .merge(staff_df[["staff_id", "store_id"]], on="staff_id")
)

# 6. Kiralama ve mağaza bilgilerini birleştir
merged_df = (
    least_revenue_film_df[["rental_id", "title"]]
    .merge(rental_staff_store_df, on="rental_id")
)

# 7. store_id ve film adına göre grupla, kiralama sayısını hesapla
final_df = (
    merged_df
    .groupby(["store_id", "title"])
    .size()
    .reset_index(name="rental_count")
)

print(final_df)

   store_id        title  rental_count
0         1  TEXAS WATCH             3
1         2  TEXAS WATCH             3


### Soru 51: Müşterilerin kiraladıkları filmlerin toplam kiralama süresi ne kadar?

In [158]:
# 1. customer ve rental tablolarını müşteri_id üzerinden birleştir
customer_rental_df = customer_df.merge(rental_df, on="customer_id")

# 2. Tarihleri datetime formatına çevir
customer_rental_df["rental_date"] = pd.to_datetime(customer_rental_df["rental_date"])
customer_rental_df["return_date"] = pd.to_datetime(customer_rental_df["return_date"])

# 3. Kira süresi (gün olarak) hesapla
customer_rental_df["rental_duration"] = (customer_rental_df["return_date"] - customer_rental_df["rental_date"]).dt.total_seconds() / (24 * 3600)

# 4. Müşteri adına göre toplam kiralama süresini hesapla
total_rental_duration_df = (
    customer_rental_df
    .groupby(["first_name", "last_name"])["rental_duration"]
    .sum()
    .reset_index(name="total_date")
)

print(total_rental_duration_df)


    first_name last_name  total_date
0        AARON     SELBY  120.398611
1         ADAM     GOOCH  116.220833
2       ADRIAN     CLARY  101.916667
3        AGNES    BISHOP  111.152778
4         ALAN      KAHN  143.023611
..         ...       ...         ...
594     WILLIE   MARKHAM  125.833333
595      WILMA  RICHARDS  117.254861
596    YOLANDA    WEAVER  139.776389
597     YVONNE   WATKINS   97.145139
598    ZACHARY      HITE  169.749306

[599 rows x 3 columns]


### Soru 52: En çok kiralanan türdeki filmler hangileri?

In [166]:
# 1. Film ve Film_Category birleşimi
film_film_category = film_df.merge(film_category_df, on="film_id")

# 2. Film_Category ve Category birleşimi
film_category_full = film_film_category.merge(category_df, on="category_id")

# 3. film_df + inventory_df + rental_df birleşimi
film_inventory_rental = (
    film_df
    .merge(inventory_df[["inventory_id", "film_id"]], on="film_id")
    .merge(rental_df[["rental_id", "inventory_id"]], on="inventory_id")
)

# 4. Tür ve film adına göre kiralama sayısını bulmak için birleşim
full_df = film_category_full.merge(film_inventory_rental, on="film_id", suffixes=('_cat', '_inv'))

# 5. Tür (genre) ve film adına göre kiralama sayısı
rental_counts = (
    full_df
    .groupby(["name", "title_cat"])
    .agg(rental_count=("rental_id", "count"))
    .reset_index()
)

# 6. En çok kiralanan filmi seç
most_rented = rental_counts.sort_values("rental_count", ascending=False).head(1)

# 7. Kolon isimlerini SQL sorgusuna uyarlayalım
most_rented = most_rented.rename(columns={"name": "genre"})

print(most_rented)

      genre           title_cat  rental_count
909  Travel  BUCKET BROTHERHOOD            34


### Soru 53: En az kiralanan türdeki filmler hangileri?

In [168]:
film_min = film_df[["film_id", "title"]]
film_category_min = film_category_df[["film_id", "category_id"]]
category_min = category_df[["category_id", "name"]]

film_film_category = film_min.merge(film_category_min, on="film_id")
film_category_full = film_film_category.merge(category_min, on="category_id")

print(film_category_full.columns)  # 'film_id', 'title', 'category_id', 'name' olmalı

film_inventory = inventory_df.merge(film_category_full[["film_id", "title", "name"]], on="film_id")

full_rental = rental_df.merge(film_inventory, on="inventory_id")

print(full_rental.columns)  # 'title', 'name' 

rental_counts = (
    full_rental
    .groupby(["name", "title"])
    .agg(rental_count=("rental_id", "count"))
    .reset_index()
)

# sorted_df'den önce sıralıyoruz
sorted_df = rental_counts.sort_values(by=["rental_count"])

# minimum rental_count değerini alıyoruz
min_count = sorted_df['rental_count'].min()

# minimum rental_count olan satırları filtreliyoruz
subset = sorted_df[sorted_df['rental_count'] == min_count]

# index'e göre en küçük satırı alıyoruz (DataFrame olarak)
first_row_df = subset.loc[[subset.index.min()]] 
print("******************")
print(first_row_df)


Index(['film_id', 'title', 'category_id', 'name'], dtype='object')
Index(['rental_id', 'rental_date', 'inventory_id', 'customer_id',
       'return_date', 'staff_id', 'film_id', 'store_id', 'last_update',
       'title', 'name'],
      dtype='object')
******************
            name           title  rental_count
315  Documentary  HARDLY ROBBERS             4


### Soru 54: Her müşteri için toplam ödeme miktarını bulun.

In [171]:
# 1. rental ile payment'i rental_id üzerinden birleştir
rental_payment_df = rental_df.merge(payment_df, on="rental_id")

# 2. rental_payment_df ile customer'ı customer_id üzerinden birleştir
customer_payment_df = rental_payment_df.merge(customer_df, left_on="customer_id_x", right_on="customer_id")

# 3. Müşteri bazında toplam ödeme (amount) hesapla
total_payment_per_customer = (
    customer_payment_df
    .groupby(["first_name", "last_name"])["amount"]
    .sum()
    .reset_index(name="total_payment")
)

print(total_payment_per_customer)


    first_name last_name  total_payment
0        AARON     SELBY         110.76
1         ADAM     GOOCH         101.78
2       ADRIAN     CLARY          74.81
3        AGNES    BISHOP          98.77
4         ALAN      KAHN         124.74
..         ...       ...            ...
594     WILLIE   MARKHAM         101.75
595      WILMA  RICHARDS          91.80
596    YOLANDA    WEAVER         110.73
597     YVONNE   WATKINS          92.79
598    ZACHARY      HITE         146.69

[599 rows x 3 columns]


### Soru 55: Hangi filmler en uzun süre kiralanmış?

In [68]:
# 1. inventory ve film'i film_id ile birleştir
film_inventory = inventory_df.merge(film_df[['film_id', 'title']], on='film_id')

# 2. rental ile film_inventory'yi inventory_id ile birleştir
rental_film = rental_df.merge(film_inventory, on='inventory_id')

# 3. Kiralama süresini hesapla (return_date - rental_date) gün olarak
rental_film['rental_days'] = (
    pd.to_datetime(rental_film['return_date']) - pd.to_datetime(rental_film['rental_date'])
).dt.total_seconds() / (24*3600)  # saniyeyi güne çeviriyoruz (float)


# 4. Her film için maksimum kiralama süresini bul
max_rental_days_per_film = (
    rental_film
    .groupby('title')['rental_days']
    .max()
    .reset_index(name='max_rental_days')
    .sort_values(by='max_rental_days', ascending=False)
)
# max_rental_days'e göre, eşitse index'e göre sıralama
max_rental_days_per_film = max_rental_days_per_film.sort_values(
    by=['max_rental_days'], ascending=True
).reset_index()  # index sırasını da kullanabilmek için ekliyoruz

# index'e göre sıralama da ekleyerek çoklu sıralama yapalım
max_rental_days_per_film = max_rental_days_per_film.sort_values(
    by=['max_rental_days', 'index'], ascending=[False, False]
).drop(columns='index')  

print(max_rental_days_per_film)

                  title  max_rental_days
956  HOLOCAUST HIGHBALL         9.249306
957     HIGHBALL POTTER         9.249306
953        TRAMP OTHERS         9.248611
955          PANIC CLUB         9.248611
952          MASK PEACH         9.248611
..                  ...              ...
4            SQUAD FISH         5.908333
3     CONSPIRACY SPIRIT         5.809722
2           GLORY TRACY         5.120833
1        TRAFFIC HOBBIT         4.977778
0     DUFFEL APOCALYPSE         4.137500

[958 rows x 2 columns]


### Soru 56: Hangi filmler en kısa süre kiralanmış?

In [71]:
# 1. inventory ile film'i film_id üzerinden birleştir
film_inventory = inventory_df.merge(film_df[['film_id', 'title']], on='film_id')

# 2. rental ile film_inventory'yi inventory_id üzerinden birleştir
rental_film = rental_df.merge(film_inventory, on='inventory_id')

# 3. Kiralama süresi hesapla
rental_film['rental_days'] = (
    pd.to_datetime(rental_film['return_date']) - pd.to_datetime(rental_film['rental_date'])
).dt.total_seconds() / (24 * 3600)

# 4. Her film için minimum kiralama süresi
min_rental_days_per_film = (
    rental_film
    .groupby('title')['rental_days']
    .min()
    .reset_index(name='min_rental_days')
    .sort_values(by='min_rental_days', ascending=True)
)
# max_rental_days'e göre, eşitse index'e göre sıralama
min_rental_days_per_film = min_rental_days_per_film.sort_values(
    by=['min_rental_days'], ascending=True
).reset_index()  # index sırasını da kullanabilmek için ekliyoruz

# index'e göre sıralama da ekleyerek çoklu sıralama yapalım
min_rental_days_per_film = min_rental_days_per_film.sort_values(
    by=['min_rental_days', 'index'], ascending=[True, True]
).drop(columns='index')  
print(min_rental_days_per_film)

                 title  min_rental_days
0    CURTAIN VIDEOTAPE         0.750000
1      DISCIPLE MOTHER         0.750000
2         ROAD ROXANNE         0.750694
3       FIDELITY DEVIL         0.751389
6       HEAVEN FREEDOM         0.751389
..                 ...              ...
953        MIXED DOORS         3.240972
954         EARLY HOME         3.807639
955    PANKY SUBMARINE         3.827083
956         WAGON JAWS         3.859028
957       ZHIVAGO CORE         3.875000

[958 rows x 2 columns]


### Soru 57: Müşterilerin kiraladığı filmler için ortalama ödeme miktarını bulun.

In [73]:
# 1. rental ile payment'i rental_id üzerinden birleştir
rental_payment_df = rental_df.merge(payment_df, on="rental_id")

# 2. customer ile birleştir
customer_payment_df = rental_payment_df.merge(customer_df, left_on="customer_id_x", right_on="customer_id")

# 3. Müşteri bazında ortalama ödeme hesapla
avg_payment_per_customer = (
    customer_payment_df
    .groupby(["first_name", "last_name"])["amount"]
    .mean()
    .reset_index(name="avg_payment")
)

print(avg_payment_per_customer)

    first_name last_name  avg_payment
0        AARON     SELBY     4.615000
1         ADAM     GOOCH     4.626364
2       ADRIAN     CLARY     3.937368
3        AGNES    BISHOP     4.294348
4         ALAN      KAHN     4.797692
..         ...       ...          ...
594     WILLIE   MARKHAM     4.070000
595      WILMA  RICHARDS     4.590000
596    YOLANDA    WEAVER     4.101111
597     YVONNE   WATKINS     4.418571
598    ZACHARY      HITE     4.731935

[599 rows x 3 columns]


### Soru 58: En çok kiralanan filmlerin ortalama kiralama süresi nedir?

In [82]:
# inventory ile film'i birleştir
film_inventory = inventory_df.merge(film_df[['film_id', 'title']], on='film_id')

# rental ile film_inventory'yi birleştir
rental_film = rental_df.merge(film_inventory, on='inventory_id')

# Kiralama süresi
rental_film['rental_days'] = (
    pd.to_datetime(rental_film['return_date']) - pd.to_datetime(rental_film['rental_date'])
).dt.total_seconds() / (24*3600)

# Ortalama kiralama süresi ve kiralama sayısı
avg_days_count = (
    rental_film
    .groupby('title')
    .agg(
        avg_rental_days=('rental_days', 'mean'),
        rental_count=('rental_id', 'count')
    )
    .sort_values(by='rental_count', ascending=False)
    .head(10)
    .reset_index()
)
print(avg_days_count)

                 title  avg_rental_days  rental_count
0   BUCKET BROTHERHOOD         4.948448            34
1     ROCKETEER MOTHER         5.386069            33
2        SCALAWAG DUCK         4.069640            32
3  RIDGEMONT SUBMARINE         6.010909            32
4       GRIT CLOCKWORK         5.340864            32
5       FORWARD TEMPLE         5.595247            32
6       JUGGLER HARDLY         5.509901            32
7        APACHE DIVINE         4.269064            31
8      RUSH GOODFELLAS         4.481855            31
9            WIFE TURN         4.792876            31


### Soru 59: En az kiralanan filmlerin ortalama kiralama süresi nedir?(*)

In [175]:
import pandas as pd

# 1. film ve inventory'yi birleştir (film_id ile)
film_inventory = inventory_df.merge(film_df[['film_id', 'title']], on='film_id')

# 2. rental ile film_inventory'yi birleştir (inventory_id ile)
rental_film = rental_df.merge(film_inventory, on='inventory_id')

# 3. Kiralama süresini gün olarak hesapla
rental_film['rental_days'] = (
    pd.to_datetime(rental_film['return_date']) - pd.to_datetime(rental_film['rental_date'])
).dt.total_seconds() / (24 * 3600)  # float gün

# 4. Group by title: kiralama sayısı ve ortalama gün
summary = (
    rental_film
    .groupby('title')
    .agg(
        rental_count=('rental_id', 'count'),
        avg_rental_days=('rental_days', 'mean')
    )
    .reset_index()
)

# 5. rental_count’a göre artan sırala ve ilk 10 filmi al
least_rented_10 = summary.sort_values(by='rental_count', ascending=True).head(10)

print(least_rented_10)

                 title  rental_count  avg_rental_days
866        TRAIN BUNCH             4         3.349306
378     HARDLY ROBBERS             4         6.948438
558        MIXED DOORS             4         5.953299
315  FREEDOM CLEOPATRA             5         2.959722
341        GLORY TRACY             5         3.678472
865     TRAFFIC HOBBIT             5         3.401528
669       PRIVATE DROP             5         4.949583
293       FEVER EMPIRE             5         5.755694
87    BRAVEHEART HUMAN             5         4.019861
100        BUNCH MINDS             5         5.524444


### Soru 60: En çok kazanç sağlayan müşterilerin ortalama ödeme miktarı nedir?

In [89]:
# 1. rental ile payment'ı rental_id üzerinden birleştir
rental_payment_df = rental_df.merge(payment_df, on="rental_id")

# 2. rental_payment_df ile customer'ı customer_id üzerinden birleştir
customer_payment_df = rental_payment_df.merge(customer_df, left_on="customer_id_x", right_on="customer_id")

# 3. Müşteri bazında ortalama ve toplam ödeme hesapla
summary = (
    customer_payment_df
    .groupby(["first_name", "last_name"])
    .agg(
        avg_payment=("amount", "mean"),
        total_payment=("amount", "sum")
    )
    .reset_index()
)

# 4. Toplam ödemeye göre azalan sırala ve ilk 10 müşteriyi al
top_10_customers = summary.sort_values(by="total_payment", ascending=False).head(10)

# 5. Sadece istenen kolonları yazdır
print(top_10_customers[["first_name", "last_name", "avg_payment"]])

    first_name last_name  avg_payment
318       KARL      SEAL     4.923333
175    ELEANOR      HUNT     4.707391
105      CLARA      SHAW     4.656667
389     MARION    SNYDER     4.990000
474     RHONDA   KENNEDY     4.990000
556      TOMMY   COLLAZO     4.911053
590     WESLEY      BULL     4.440000
551        TIM      CARY     4.502821
379     MARCIA      DEAN     4.180476
21         ANA   BRADLEY     5.137059


### Soru 61: En az kazanç sağlayan müşterilerin ortalama ödeme miktarı nedir?

In [92]:
# 1. rental ile payment'ı rental_id üzerinden birleştir
rental_payment_df = rental_df.merge(payment_df, on="rental_id")

# 2. rental_payment_df ile customer'ı customer_id üzerinden birleştir
customer_payment_df = rental_payment_df.merge(customer_df, left_on="customer_id_x", right_on="customer_id")

# 3. Müşteri bazında ortalama ve toplam ödeme hesapla
summary = (
    customer_payment_df
    .groupby(["first_name", "last_name"])
    .agg(
        avg_payment=("amount", "mean"),
        total_payment=("amount", "sum")
    )
    .reset_index()
)

# 4. Toplam ödemeye göre artan sırala ve ilk 10 müşteriyi al
bottom_10_customers = summary.sort_values(by="total_payment", ascending=True).head(10)

# 5. İstenen kolonları yazdır
print(bottom_10_customers[["first_name", "last_name", "avg_payment"]])

    first_name last_name  avg_payment
83    CAROLINE    BOWMAN     3.390000
351      LEONA    OBRIEN     3.632857
71       BRIAN     WYMAN     4.406667
296     JOHNNY    TURPIN     3.042632
33       ANNIE   RUSSELL     3.267778
319  KATHERINE    RIVERA     4.204286
550    TIFFANY    JORDAN     4.275714
28       ANITA   MORALES     4.190000
401     MATTIE   HOFFMAN     2.944545
334       KIRK   STCLAIR     3.411053


### Soru 62: Mağazalardaki toplam kiralama süresini bulun.

In [94]:
# 1. staff ile rental'ı staff_id üzerinden birleştir
staff_rental = staff_df.merge(rental_df, on="staff_id")

# 2. staff_rental ile store'u store_id üzerinden birleştir
store_rental = staff_rental.merge(store_df, on="store_id")

# 3. Kiralama sürelerini gün olarak hesapla
store_rental['rental_days'] = (
    pd.to_datetime(store_rental['return_date']) - pd.to_datetime(store_rental['rental_date'])
).dt.total_seconds() / (24 * 3600)  # float gün

# 4. Store bazında toplam kiralama süresi hesapla
total_rental_days_per_store = (
    store_rental
    .groupby('store_id')['rental_days']
    .sum()
    .reset_index(name='total_rental_days')
)

print(total_rental_days_per_store)

   store_id  total_rental_days
0         1       39878.755556
1         2       39828.009028


### Soru 63: En uzun süre kiralanan film hangi mağazada kiralanmış?

In [106]:
# staff ve store'da ortak olan gereksiz sütunları düşürelim
staff_small = staff_df.drop(columns=['last_update'])
store_small = store_df.drop(columns=['last_update'])

# 1. staff ile store'u store_id üzerinden birleştir
staff_store = staff_small.merge(store_small, on='store_id')

# 2. rental ile staff_store'ı staff_id üzerinden birleştir
rental_staff_store = rental_df.merge(staff_store, on='staff_id')

# 3. inventory ile rental_staff_store'u inventory_id üzerinden birleştir
rental_inventory = rental_staff_store.merge(inventory_df, on='inventory_id')

# 4. film ile rental_inventory'yi film_id üzerinden birleştir
rental_film = rental_inventory.merge(film_df[['film_id', 'title']], on='film_id')

# 5. Kiralama süresini gün olarak hesapla
rental_film['rental_days'] = (
    pd.to_datetime(rental_film['return_date']) - pd.to_datetime(rental_film['rental_date'])
).dt.total_seconds() / (24 * 3600)

# 6. store_id ve film.title göre gruplama yap, maksimum kiralama süresini bul
max_rental = (
    rental_film
    .groupby(['store_id_x', 'title'])['rental_days']
    .max()
    .reset_index()
    .sort_values(by='rental_days', ascending=False)
    .head(1)
)

print(max_rental)

     store_id_x               title  rental_days
400           1  HOLOCAUST HIGHBALL     9.249306


### Soru 64: En kısa süre kiralanan film hangi mağazada kiralanmış?

In [109]:
# staff ve store'da ortak olan gereksiz sütunları düşürelim
staff_small = staff_df.drop(columns=['last_update'])
store_small = store_df.drop(columns=['last_update'])

# 1. staff ile store'u store_id üzerinden birleştir
staff_store = staff_small.merge(store_small, on='store_id')

# 2. rental ile staff_store'ı staff_id üzerinden birleştir
rental_staff_store = rental_df.merge(staff_store, on='staff_id')

# 3. inventory ile rental_staff_store'u inventory_id üzerinden birleştir
rental_inventory = rental_staff_store.merge(inventory_df, on='inventory_id')

# 4. film ile rental_inventory'yi film_id üzerinden birleştir
rental_film = rental_inventory.merge(film_df[['film_id', 'title']], on='film_id')

# 5. Kiralama süresini gün olarak hesapla
rental_film['rental_days'] = (
    pd.to_datetime(rental_film['return_date']) - pd.to_datetime(rental_film['rental_date'])
).dt.total_seconds() / (24 * 3600)
#6. store_id ve title'a göre grupla, maksimum kiralama süresini hesapla
grouped = rental_film.groupby(['store_id_x', 'title'])['rental_days'].max().reset_index()

# 7. En küçük max_rental_days değerine göre sırala ve ilk kaydı al
result = grouped.sort_values(by='rental_days', ascending=True).head(1)

print(result)

     store_id_x              title  rental_days
567           1  MONTEZUMA COMMAND     0.820139


### Soru 65: Her filmin ortalama kiralama süresi nedir?

In [111]:
# 1. film ile inventory'yi film_id üzerinden birleştir
film_inventory = film_df.merge(inventory_df, on='film_id')

# 2. rental ile film_inventory'yi inventory_id üzerinden birleştir
rental_film = rental_df.merge(film_inventory, on='inventory_id')

# 3. Kiralama süresini gün olarak hesapla (float)
rental_film['rental_days'] = (
    pd.to_datetime(rental_film['return_date']) - pd.to_datetime(rental_film['rental_date'])
).dt.total_seconds() / (24 * 3600)

# 4. Film başına ortalama kiralama süresini hesapla
avg_rental_days = (
    rental_film
    .groupby('title')['rental_days']
    .mean()
    .reset_index(name='AVG_rental_days')
)

print(avg_rental_days)

                 title  AVG_rental_days
0     ACADEMY DINOSAUR         4.997128
1       ACE GOLDFINGER         5.642708
2     ADAPTATION HOLES         3.465625
3     AFFAIR PREJUDICE         4.758333
4          AFRICAN EGG         7.106629
..                 ...              ...
953     YOUNG LANGUAGE         4.624405
954         YOUTH KICK         5.562963
955       ZHIVAGO CORE         5.950087
956  ZOOLANDER FICTION         5.542647
957          ZORRO ARK         4.539180

[958 rows x 2 columns]


### Soru 66: Hangi filmler en çok kiralanan kategoride?

In [180]:
import pandas as pd

# 1. category + film_category (category_id ile)
cat_film_cat = pd.merge(category_df, film_category_df, on='category_id')

# 2. film_category + film (film_id ile)
cat_film = pd.merge(cat_film_cat, film_df, on='film_id')

# 3. film + inventory (film_id ile)
film_inventory = pd.merge(cat_film, inventory_df[['inventory_id', 'film_id']], on='film_id')

# 4. inventory + rental (inventory_id ile)
full_df = pd.merge(film_inventory, rental_df, on='inventory_id')

# 5. genre (category name), film title ve kiralama sayısını hesapla
result = (
    full_df
    .groupby(['name', 'title'])
    .agg(rental_count=('rental_id', 'count'))
    .reset_index()
    .rename(columns={'name': 'genre'})
    .sort_values(by='rental_count', ascending=False)
)
# Orijinal indexi yeni bir sütun olarak ekle
result = result.reset_index()  # Böylece 'index' sütunu oluşur

# rental_count azalan, index artan olarak sırala
result = result.sort_values(by=['rental_count', 'index'], ascending=[False, True])

# İstersen sıfırlanmış index'i kaldır
result = result.reset_index(drop=True)
print(result)

     index        genre               title  rental_count
0      909       Travel  BUCKET BROTHERHOOD            34
1      536      Foreign    ROCKETEER MOTHER            33
2       93    Animation      JUGGLER HARDLY            32
3      568        Games      FORWARD TEMPLE            32
4      572        Games      GRIT CLOCKWORK            32
..     ...          ...                 ...           ...
953    872       Sports  MUSSOLINI SPOILERS             5
954    951       Travel      TRAFFIC HOBBIT             5
955    315  Documentary      HARDLY ROBBERS             4
956    525      Foreign         MIXED DOORS             4
957    656       Horror         TRAIN BUNCH             4

[958 rows x 4 columns]


### Soru 67: Hangi filmler en az kiralanan kategoride?

In [183]:
result = result.sort_values(by=['rental_count', 'index'], ascending=[True, True])
print(result)

     index        genre                title  rental_count
955    315  Documentary       HARDLY ROBBERS             4
956    525      Foreign          MIXED DOORS             4
957    656       Horror          TRAIN BUNCH             4
941    141     Children      FULL FLATLINERS             5
942    189     Classics    CONSPIRACY SPIRIT             5
..     ...          ...                  ...           ...
4      572        Games       GRIT CLOCKWORK            32
5      701        Music        SCALAWAG DUCK            32
6      757          New  RIDGEMONT SUBMARINE            32
1      536      Foreign     ROCKETEER MOTHER            33
0      909       Travel   BUCKET BROTHERHOOD            34

[958 rows x 4 columns]


### Soru 68: Hangi mağazalarda en çok film kiralanmış?

In [119]:
# 1. staff ile store'u store_id üzerinden birleştir
staff_store = staff_df.merge(store_df, on='store_id')

# 2. rental ile staff_store'u staff_id üzerinden birleştir
rental_staff_store = rental_df.merge(staff_store, on='staff_id')

# 3. store_id bazında kiralama sayısını (rental_id sayısı) hesapla
rental_count_per_store = (
    rental_staff_store
    .groupby('store_id')['rental_id']
    .count()
    .reset_index(name='rental_count')
    .sort_values(by='rental_count', ascending=False)
)

print(rental_count_per_store)

   store_id  rental_count
0         1          8040
1         2          8004


### Soru 69: Hangi mağazalarda en az film kiralanmış?

In [121]:
# 1. staff ile store'u store_id üzerinden birleştir
staff_store = staff_df.merge(store_df, on='store_id')

# 2. rental ile staff_store'u staff_id üzerinden birleştir
rental_staff_store = rental_df.merge(staff_store, on='staff_id')

# 3. store_id bazında kiralama sayısını (rental_id sayısı) hesapla
rental_count_per_store = (
    rental_staff_store
    .groupby('store_id')['rental_id']
    .count()
    .reset_index(name='rental_count')
    .sort_values(by='rental_count', ascending=True)  # Artan sırada sıralama
)

print(rental_count_per_store)

   store_id  rental_count
1         2          8004
0         1          8040


### Soru 70: Hangi aktörler en çok filmde rol almış?

In [129]:
# 1. actor ile film_actor'ü actor_id üzerinden birleştir
actor_films = actor_df.merge(film_actor_df, on='actor_id')

# 2. actor bazında film sayısını hesapla
film_count_per_actor = (
    actor_films
    .groupby(['first_name', 'last_name'])['film_id']
    .count()
    .reset_index(name='film_count')
)

# Orijinal index'i yeni bir sütun olarak ekle
film_count_per_actor = film_count_per_actor.reset_index()

# Şimdi film_count azalan, index artan olacak şekilde sırala
film_count_per_actor = film_count_per_actor.sort_values(
    by=['film_count', 'index'], ascending=[False, True]
).reset_index(drop=True)

print(film_count_per_actor)

     index first_name  last_name  film_count
0      180      SUSAN      DAVIS          54
1       65       GINA  DEGENERES          42
2      190     WALTER       TORN          41
3      123       MARY     KEITEL          40
4      125    MATTHEW     CARREY          39
..     ...        ...        ...         ...
194    177      SISSY   SOBIESKI          18
195    103      JULIA  ZELLWEGER          16
196     99       JUDY       DEAN          15
197    101      JULIA    FAWCETT          15
198     51      EMILY        DEE          14

[199 rows x 4 columns]


### Soru 71: Hangi aktörler en az filmde rol almış?

In [132]:
# 1. actor ile film_actor'ü actor_id üzerinden birleştir
actor_films = actor_df.merge(film_actor_df, on='actor_id')

# 2. actor bazında film sayısını hesapla
film_count_per_actor = (
    actor_films
    .groupby(['first_name', 'last_name'])['film_id']
    .count()
    .reset_index(name='film_count')
)

# Orijinal index'i yeni bir sütun olarak ekle
film_count_per_actor = film_count_per_actor.reset_index()

# Şimdi film_count artan, index artan olacak şekilde sırala
film_count_per_actor = film_count_per_actor.sort_values(
    by=['film_count', 'index'], ascending=[True, True]
).reset_index(drop=True)

print(film_count_per_actor)

     index first_name  last_name  film_count
0       51      EMILY        DEE          14
1       99       JUDY       DEAN          15
2      101      JULIA    FAWCETT          15
3      103      JULIA  ZELLWEGER          16
4        0       ADAM      GRANT          18
..     ...        ...        ...         ...
194    125    MATTHEW     CARREY          39
195    123       MARY     KEITEL          40
196    190     WALTER       TORN          41
197     65       GINA  DEGENERES          42
198    180      SUSAN      DAVIS          54

[199 rows x 4 columns]


### Soru 72: Hangi aktörlerin oynadığı filmler en çok kiralanmış?

In [134]:
# 1. actor ile film_actor'ü actor_id üzerinden birleştir
actor_films = actor_df.merge(film_actor_df, on='actor_id')

# 2. film_actor ile film'i film_id üzerinden birleştir
film_inventory = film_df.merge(inventory_df, on='film_id')

# 3. film_inventory ile rental'ı inventory_id üzerinden birleştir
rental_inventory = rental_df.merge(film_inventory, on='inventory_id')

# 4. actor_films ile rental_inventory'yi film_id ve actor_id üzerinden birleştir
# Öncelikle actor_films ile rental_inventory'nin film_id ve actor_id sütunlarını birleştirelim
actor_rental = actor_films.merge(rental_inventory, on='film_id')

# 5. Aktör bazında toplam kiralama sayısını hesapla
rental_count_per_actor = (
    actor_rental
    .groupby(['first_name', 'last_name'])['rental_id']
    .count()
    .reset_index(name='rental_count')
    .sort_values(by='rental_count', ascending=False)
)

print(rental_count_per_actor)

    first_name    last_name  rental_count
180      SUSAN        DAVIS           825
65        GINA    DEGENERES           753
125    MATTHEW       CARREY           678
123       MARY       KEITEL           674
8       ANGELA  WITHERSPOON           654
..         ...          ...           ...
99        JUDY         DEAN           255
101      JULIA      FAWCETT           255
177      SISSY     SOBIESKI           235
103      JULIA    ZELLWEGER           221
51       EMILY          DEE           216

[199 rows x 3 columns]


### Soru 73: Hangi aktörlerin oynadığı filmler en az kiralanmış?

In [136]:
# 1. actor ile film_actor'ü actor_id üzerinden birleştir
actor_films = actor_df.merge(film_actor_df, on='actor_id')

# 2. film_actor ile film'i film_id üzerinden birleştir
film_inventory = film_df.merge(inventory_df, on='film_id')

# 3. film_inventory ile rental'ı inventory_id üzerinden birleştir
rental_inventory = rental_df.merge(film_inventory, on='inventory_id')

# 4. actor_films ile rental_inventory'yi film_id ve actor_id üzerinden birleştir
# Öncelikle actor_films ile rental_inventory'nin film_id ve actor_id sütunlarını birleştirelim
actor_rental = actor_films.merge(rental_inventory, on='film_id')

# 5. Aktör bazında toplam kiralama sayısını hesapla
rental_count_per_actor = (
    actor_rental
    .groupby(['first_name', 'last_name'])['rental_id']
    .count()
    .reset_index(name='rental_count')
    .sort_values(by='rental_count', ascending=True)
)

print(rental_count_per_actor)

    first_name    last_name  rental_count
51       EMILY          DEE           216
103      JULIA    ZELLWEGER           221
177      SISSY     SOBIESKI           235
99        JUDY         DEAN           255
101      JULIA      FAWCETT           255
..         ...          ...           ...
8       ANGELA  WITHERSPOON           654
123       MARY       KEITEL           674
125    MATTHEW       CARREY           678
65        GINA    DEGENERES           753
180      SUSAN        DAVIS           825

[199 rows x 3 columns]


### Soru 74: Hangi kategorilerde en fazla aktör oynamış?

In [145]:
# Gereksiz sütunları düşür
category_small = category_df.drop(columns=['last_update'])
film_category_small = film_category_df.drop(columns=['last_update'])

# 1. category ile film_category'yi category_id üzerinden birleştir (name sütununu alıyoruz)
category_film_cat = category_small.merge(film_category_small, on='category_id')

# 2. film_category ile film'i film_id üzerinden birleştir (sadece film_id alınabilir)
film_cat = category_film_cat.merge(film_df[['film_id']], on='film_id')

# 3. film_actor ile film_id, actor_id alınarak birleştir
film_actor_small = film_actor_df[['film_id', 'actor_id']]

# 4. film_cat ile film_actor'ü film_id üzerinden birleştir
film_cat_actor = film_cat.merge(film_actor_small, on='film_id')

# 5. genre (category name) bazında benzersiz aktör sayısını hesapla
actor_count_per_genre = (
    film_cat_actor
    .groupby('name_x')['actor_id']
    .nunique()
    .reset_index(name='actor_count')
    .sort_values(by='actor_count', ascending=False)
    .rename(columns={'name_x': 'genre'})
)

print(actor_count_per_genre)

          genre  actor_count
14       Sports          182
8       Foreign          175
12          New          169
5   Documentary          168
13       Sci-Fi          167
0        Action          166
1     Animation          166
15       Travel          166
7        Family          164
2      Children          163
3      Classics          162
6         Drama          162
10       Horror          156
9         Games          150
4        Comedy          147
11        Music          144


### Soru 75: Hangi kategorilerde en az aktör oynamış?

In [147]:
# Gereksiz sütunları düşür
category_small = category_df.drop(columns=['last_update'])
film_category_small = film_category_df.drop(columns=['last_update'])

# 1. category ile film_category'yi category_id üzerinden birleştir (name sütununu alıyoruz)
category_film_cat = category_small.merge(film_category_small, on='category_id')

# 2. film_category ile film'i film_id üzerinden birleştir (sadece film_id alınabilir)
film_cat = category_film_cat.merge(film_df[['film_id']], on='film_id')

# 3. film_actor ile film_id, actor_id alınarak birleştir
film_actor_small = film_actor_df[['film_id', 'actor_id']]

# 4. film_cat ile film_actor'ü film_id üzerinden birleştir
film_cat_actor = film_cat.merge(film_actor_small, on='film_id')

# 5. genre (category name) bazında benzersiz aktör sayısını hesapla
actor_count_per_genre = (
    film_cat_actor
    .groupby('name_x')['actor_id']
    .nunique()
    .reset_index(name='actor_count')
    .sort_values(by='actor_count', ascending=True)
    .rename(columns={'name_x': 'genre'})
)

print(actor_count_per_genre)

          genre  actor_count
11        Music          144
4        Comedy          147
9         Games          150
10       Horror          156
6         Drama          162
3      Classics          162
2      Children          163
7        Family          164
1     Animation          166
0        Action          166
15       Travel          166
13       Sci-Fi          167
5   Documentary          168
12          New          169
8       Foreign          175
14       Sports          182


### Soru 76: Hangi kategorilerde oynayan filmler en çok kiralanmış?

In [150]:
# 1. Gereksiz sütunları düşürelim
category_small = category_df.drop(columns=['last_update'])
film_category_small = film_category_df.drop(columns=['last_update'])

# 2. category ile film_category'yi category_id üzerinden birleştir
category_filmcat = category_small.merge(film_category_small, on='category_id')

# 3. film ile category_filmcat'i film_id üzerinden birleştir
category_film = category_filmcat.merge(film_df[['film_id']], on='film_id')

# 4. inventory ile film'i film_id üzerinden birleştir
film_inventory = category_film.merge(inventory_df, on='film_id')

# 5. rental ile inventory'yi inventory_id üzerinden birleştir
rental_full = film_inventory.merge(rental_df[['inventory_id', 'rental_id']], on='inventory_id')

# 6. genre (kategori adı) bazında toplam kiralama sayısını hesapla
genre_rental_counts = (
    rental_full
    .groupby('name_x')['rental_id']
    .count()
    .reset_index(name='rental_count')
    .sort_values(by='rental_count', ascending=False)
    .rename(columns={'name_x': 'genre'})
)
print(genre_rental_counts)

          genre  rental_count
14       Sports          1179
1     Animation          1166
0        Action          1112
13       Sci-Fi          1101
7        Family          1096
6         Drama          1060
5   Documentary          1050
8       Foreign          1033
9         Games           969
2      Children           945
4        Comedy           941
12          New           940
3      Classics           939
10       Horror           846
15       Travel           837
11        Music           830


### Soru 77: Hangi kategorilerde oynayan filmler en az kiralanmış?

In [152]:
genre_rental_counts = (
    rental_full
    .groupby('name_x')['rental_id']
    .count()
    .reset_index(name='rental_count')
    .sort_values(by='rental_count', ascending=True) #artan sırada
    .rename(columns={'name_x': 'genre'})
)
print(genre_rental_counts)

          genre  rental_count
11        Music           830
15       Travel           837
10       Horror           846
3      Classics           939
12          New           940
4        Comedy           941
2      Children           945
9         Games           969
8       Foreign          1033
5   Documentary          1050
6         Drama          1060
7        Family          1096
13       Sci-Fi          1101
0        Action          1112
1     Animation          1166
14       Sports          1179


### Soru 78: En fazla kazanç sağlayan kategoriler hangileri?(*)

In [185]:
# 1. category + film_category (category_id ile)
cat_film_cat = pd.merge(category_df, film_category_df, on='category_id')

# 2. film_category + film (film_id ile)
cat_film = pd.merge(cat_film_cat, film_df, on='film_id')

# 3. film + inventory (film_id ile)
film_inventory = pd.merge(cat_film, inventory_df[['inventory_id', 'film_id']], on='film_id')

# 4. inventory + rental (inventory_id ile)
inventory_rental = pd.merge(film_inventory, rental_df[['rental_id', 'inventory_id']], on='inventory_id')

# 5. rental + payment (rental_id ile)
rental_payment = pd.merge(inventory_rental, payment_df[['rental_id', 'amount']], on='rental_id')

# 6. genre bazında toplam geliri hesapla
result = (
    rental_payment
    .groupby('name')  # category adı sütunu "name"
    .agg(total_revenue=('amount', 'sum'))
    .reset_index()
    .sort_values(by='total_revenue', ascending=False)
)

print(result)

           name  total_revenue
14       Sports        5314.21
13       Sci-Fi        4756.98
1     Animation        4656.30
6         Drama        4587.39
4        Comedy        4383.58
0        Action        4375.85
12          New        4351.62
9         Games        4281.33
8       Foreign        4270.67
7        Family        4226.07
5   Documentary        4217.52
10       Horror        3722.54
2      Children        3655.55
3      Classics        3639.59
15       Travel        3549.64
11        Music        3417.72


### Soru 79: En az kazanç sağlayan kategoriler hangileri?(*)

In [188]:
result = (
    rental_payment
    .groupby('name')  # category adı sütunu "name"
    .agg(total_revenue=('amount', 'sum'))
    .reset_index()
    .sort_values(by='total_revenue', ascending=True)
)

print(result)

           name  total_revenue
11        Music        3417.72
15       Travel        3549.64
3      Classics        3639.59
2      Children        3655.55
10       Horror        3722.54
5   Documentary        4217.52
7        Family        4226.07
8       Foreign        4270.67
9         Games        4281.33
12          New        4351.62
0        Action        4375.85
4        Comedy        4383.58
6         Drama        4587.39
1     Animation        4656.30
13       Sci-Fi        4756.98
14       Sports        5314.21


### Soru 80: En çok film kiralayan müşterilerin ortalama ödeme miktarı nedir?

In [195]:
# 1. customer + rental (customer_id ile)
customer_rental = pd.merge(customer_df, rental_df, on='customer_id')

# 2. rental + payment (rental_id ile)
rental_payment = pd.merge(customer_rental, payment_df, on='rental_id')

# 3. customer bazında ortalama ödeme ve kira sayısını hesapla
result = (
    rental_payment
    .groupby(['first_name', 'last_name'])
    .agg(
        avg_payment=('amount', 'mean'),
        rental_count=('rental_id', 'count')
    )
    .reset_index()
    .sort_values(by='rental_count', ascending=False)
    .drop(columns='rental_count')  
)

print(result)

    first_name last_name  avg_payment
175    ELEANOR      HUNT     4.707391
318       KARL      SEAL     4.923333
379     MARCIA      DEAN     4.180476
105      CLARA      SHAW     4.656667
536      TAMMY   SANDERS     3.794878
..         ...       ...          ...
28       ANITA   MORALES     4.190000
319  KATHERINE    RIVERA     4.204286
351      LEONA    OBRIEN     3.632857
550    TIFFANY    JORDAN     4.275714
71       BRIAN     WYMAN     4.406667

[599 rows x 3 columns]


### Soru 81: En az film kiralayan müşterilerin ortalama ödeme miktarı nedir?

In [197]:
result = (
    rental_payment
    .groupby(['first_name', 'last_name'])
    .agg(
        avg_payment=('amount', 'mean'),
        rental_count=('rental_id', 'count')
    )
    .reset_index()
    .sort_values(by='rental_count', ascending=True)
    .drop(columns='rental_count')  
)

print(result)

    first_name last_name  avg_payment
71       BRIAN     WYMAN     4.406667
550    TIFFANY    JORDAN     4.275714
351      LEONA    OBRIEN     3.632857
319  KATHERINE    RIVERA     4.204286
28       ANITA   MORALES     4.190000
..         ...       ...          ...
536      TAMMY   SANDERS     3.794878
105      CLARA      SHAW     4.656667
379     MARCIA      DEAN     4.180476
318       KARL      SEAL     4.923333
175    ELEANOR      HUNT     4.707391

[599 rows x 3 columns]


### Soru 82: Hangi mağazalarda hangi kategoriler en çok kiralanmış?

In [201]:
# 1. store + staff (store_id ile)
store_staff = pd.merge(store_df, staff_df, on='store_id')

# 2. staff + rental (staff_id ile)
staff_rental = pd.merge(store_staff, rental_df, on='staff_id')

# 3. rental + inventory (inventory_id ile)
rental_inventory = pd.merge(staff_rental, inventory_df[['inventory_id', 'film_id']], on='inventory_id')

# 4. inventory + film (film_id ile)
inventory_film = pd.merge(rental_inventory, film_df[['film_id']], on='film_id')

# 5. film + film_category (film_id ile)
film_film_cat = pd.merge(inventory_film, film_category_df, on='film_id')

# 6. film_category + category (category_id ile)
full_df = pd.merge(film_film_cat, category_df[['category_id', 'name']], on='category_id')

# 7. store_id ve genre bazında kiralama sayısı
result = (
    full_df
    .groupby(['store_id', 'name'])
    .agg(rental_count=('rental_id', 'count'))
    .reset_index()
    .rename(columns={'name': 'genre'})
    .sort_values(by='rental_count', ascending=False)
)

print(result)


    store_id        genre  rental_count
30         2       Sports           614
17         2    Animation           584
1          1    Animation           582
7          1       Family           565
14         1       Sports           565
0          1       Action           562
13         1       Sci-Fi           553
16         2       Action           550
29         2       Sci-Fi           548
24         2      Foreign           543
22         2        Drama           532
23         2       Family           531
6          1        Drama           528
21         2  Documentary           525
5          1  Documentary           525
9          1        Games           494
8          1      Foreign           490
12         1          New           489
2          1     Children           483
25         2        Games           475
3          1     Classics           475
4          1       Comedy           475
20         2       Comedy           466
19         2     Classics           464


### Soru 83: Hangi mağazalarda hangi kategoriler en az kiralanmış?

In [204]:
result = (
    full_df
    .groupby(['store_id', 'name'])
    .agg(rental_count=('rental_id', 'count'))
    .reset_index()
    .rename(columns={'name': 'genre'})
    .sort_values(by='rental_count', ascending=True)
)

print(result)


    store_id        genre  rental_count
27         2        Music           398
15         1       Travel           399
26         2       Horror           423
10         1       Horror           423
11         1        Music           432
31         2       Travel           438
28         2          New           451
18         2     Children           462
19         2     Classics           464
20         2       Comedy           466
25         2        Games           475
3          1     Classics           475
4          1       Comedy           475
2          1     Children           483
12         1          New           489
8          1      Foreign           490
9          1        Games           494
21         2  Documentary           525
5          1  Documentary           525
6          1        Drama           528
23         2       Family           531
22         2        Drama           532
24         2      Foreign           543
29         2       Sci-Fi           548


### Soru 84: En fazla sayıda film kiralayan mağaza hangisidir ve toplam kiralama sayısı nedir?

In [7]:
# 1. store ve staff'ı birleştir (store_id üzerinden)
store_staff = staff_df[['store_id', 'staff_id']]

# 2. rental ile staff'ı staff_id üzerinden birleştir
rental_staff = rental_df.merge(store_staff, on='staff_id')

# 3. store_id bazında kiralama sayısını hesapla
rental_counts = (
    rental_staff
    .groupby('store_id')
    .agg(total_rentals=('rental_id', 'count'))
    .reset_index()
    .sort_values(by='total_rentals', ascending=False)
    .head(1)
)
print(rental_counts)

   store_id  total_rentals
0         1           8040


### Soru 85: En az sayıda film kiralayan mağaza hangisidir ve toplam kiralama sayısı nedir?

In [9]:
# 3. store_id bazında kiralama sayısını hesapla
rental_counts = (
    rental_staff
    .groupby('store_id')
    .agg(total_rentals=('rental_id', 'count'))
    .reset_index()
    .sort_values(by='total_rentals', ascending=True)
    .head(1)
)
print(rental_counts)

   store_id  total_rentals
1         2           8004


### Soru 86: En fazla sayıda kiralama yapan müşteri hangisidir ve toplam kiralama sayısı nedir?

In [11]:
# 1. customer ile rental tablolarını customer_id üzerinden birleştir
merged_df = customer_df.merge(rental_df, on='customer_id', how='inner')

# 2. Her müşterinin (first_name, last_name) bazında kaç tane rental_id'si olduğunu say (groupby + aggregation)
total_rentals_df = merged_df.groupby(['first_name', 'last_name']).agg(total_rentals=('rental_id', 'count')).reset_index()

# 3. Kiralama sayısına göre azalan şekilde sırala
total_rentals_df = total_rentals_df.sort_values(by='total_rentals', ascending=False)

# 4. En çok kiralama yapan müşteriyi al (ilk satır)
top_customer = total_rentals_df.head(1)

print(top_customer)

    first_name last_name  total_rentals
175    ELEANOR      HUNT             46


### Soru 87: En az sayıda kiralama yapan müşteri hangisidir ve toplam kiralama sayısı nedir?

In [13]:
total_rentals_df = total_rentals_df.sort_values(by='total_rentals', ascending=True)

#  En az kiralama yapan müşteriyi al (ilk satır)
min_customer = total_rentals_df.head(1)

print(min_customer)

   first_name last_name  total_rentals
71      BRIAN     WYMAN             12


### Soru 88: En fazla sayıda kiralanan film hangisidir ve toplam kiralama sayısı nedir?

In [15]:
# 1. film ile inventory'yi film_id üzerinden birleştir
film_inventory_df = film_df.merge(inventory_df, on='film_id', how='inner')

# 2. inventory ile rental'ı inventory_id üzerinden birleştir
film_inventory_rental_df = film_inventory_df.merge(rental_df, on='inventory_id', how='inner')

# 3. film title bazında rental sayısını hesapla
rental_counts = (
    film_inventory_rental_df
    .groupby('title')
    .agg(total_rentals=('rental_id', 'count'))
    .reset_index()
)

# 4. toplam kiralama sayısına göre azalan sırala
rental_counts_sorted = rental_counts.sort_values(by='total_rentals', ascending=False)

# 5. en çok kiralanan filmi al
top_film = rental_counts_sorted.head(1)

print(top_film)

                 title  total_rentals
96  BUCKET BROTHERHOOD             34


### Soru 89: En az sayıda kiralanan film hangisidir ve toplam kiralama sayısı nedir?

In [25]:
# total_rentals artan sırada sırala
rental_counts_sorted = rental_counts.sort_values(by='total_rentals', ascending=True)

# en az kiralanan 10 filmi seç ve açıkça kopya al
min_film = rental_counts_sorted.head(10).copy()

# index'i sütun olarak ekle
min_film['index_col'] = min_film.index

# total_rentals ve index_col'a göre sıralama yap
min_film_sorted = min_film.sort_values(by=['total_rentals', 'index_col'], ascending=[True, True])

# index_col sütununu kaldır
min_film_sorted = min_film_sorted.drop(columns=['index_col'])

# ilk satırı al
first_row = min_film_sorted.iloc[0]

print(first_row)

title            HARDLY ROBBERS
total_rentals                 4
Name: 378, dtype: object


### Soru 90: En fazla kazanç sağlayan müşteri hangisidir ve toplam kazancı nedir?

In [29]:
# 1. customer ve rental tablolarını customer_id üzerinden birleştir
cust_rental_df = customer_df.merge(rental_df, on='customer_id', how='inner')

# 2. cust_rental ile payment tablolarını rental_id üzerinden birleştir
cust_rental_payment_df = cust_rental_df.merge(payment_df, on='rental_id', how='inner')

# 3. first_name ve last_name bazında toplam ödeme miktarını hesapla
total_revenue_df = (
    cust_rental_payment_df
    .groupby(['first_name', 'last_name'])
    .agg(total_revenue=('amount', 'sum'))
    .reset_index()
)

# 4. toplam gelire göre azalan şekilde sırala
total_revenue_sorted = total_revenue_df.sort_values(by='total_revenue', ascending=False)

# 5. en çok gelir getiren müşteriyi al
top_customer = total_revenue_sorted.head(1)

print(top_customer)

    first_name last_name  total_revenue
318       KARL      SEAL         221.55


### Soru 91: En az kazanç sağlayan müşteri hangisidir ve toplam kazancı nedir?

In [33]:
#toplam gelire göre artan şekilde sırala
total_revenue_sorted = total_revenue_df.sort_values(by='total_revenue', ascending=True)

#  en az gelir getiren müşteriyi al
min_customer = total_revenue_sorted.head(1)

print(min_customer)

   first_name last_name  total_revenue
83   CAROLINE    BOWMAN          50.85


### Soru 92: Hangi film, hangi kategoride en çok kazanç sağlamış?

In [43]:
# 1. film + film_category
film_fcat_df = film_df.merge(film_category_df, on='film_id', how='inner')

# 2. + category (film türü)
film_fcat_cat_df = film_fcat_df.merge(category_df, on='category_id', how='inner')

# Eğer her iki DataFrame'de de 'last_update' varsa, birinden sil (örneğin category_df)
# veya merge işleminden sonra silinir ama hata almamak için öncesinde silmek daha güvenli
film_fcat_cat_df = film_fcat_cat_df.drop(columns=['last_update'], errors='ignore')
inventory_dff = inventory_df.drop(columns=['last_update'], errors='ignore')

# 3. + inventory (film_id ortak)
film_inv_df = film_fcat_cat_df.merge(inventory_dff, on='film_id', how='inner')

# rental ile birleşme öncesi yine olası çakışmaları önleyelim
rental_dff = rental_df.drop(columns=['last_update'], errors='ignore')

# 4. + rental
film_rental_df = film_inv_df.merge(rental_dff, on='inventory_id', how='inner')

# payment ile birleşme öncesi
payment_dff = payment_df.drop(columns=['last_update'], errors='ignore')

# 5. + payment
full_df = film_rental_df.merge(payment_dff, on='rental_id', how='inner')

# 6. film title ve genre (category.name) bazında toplam amount hesapla
revenue_df = (
    full_df
    .groupby(['title', 'name'])
    .agg(total_revenue=('amount', 'sum'))
    .reset_index()
)

# 7. Azalan sırada sıralama
revenue_df = revenue_df.sort_values(by='total_revenue', ascending=False)

# 8. En çok gelir getiren film
top_film = revenue_df.head(1)

# Kolon adı düzenle
top_film = top_film.rename(columns={'name': 'genre'})

print(top_film)


                title  genre  total_revenue
841  TELEGRAPH VOYAGE  Music         231.73


### Soru 93: Hangi film, hangi kategoride en az kazanç sağlamış?

In [48]:
#  Azalan sırada sıralama
revenue_df = revenue_df.sort_values(by='total_revenue', ascending=True)

#  En az gelir getiren film
min_film = revenue_df.head(10).copy()
min_film['index_col'] = min_film.index

# Kolon adı düzenle
min_film = min_film.rename(columns={'name': 'genre'})
min_film_sorted = min_film.sort_values(by=['total_revenue', 'index_col'], ascending=[True, True])
min_film_sorted = min_film_sorted.drop(columns=['index_col'])

# ilk satırı al
first_row = min_film_sorted.iloc[0]
print(first_row)

title            OKLAHOMA JUMANJI
genre                         New
total_revenue                5.94
Name: 608, dtype: object


### Soru 94: En fazla kazanç sağlayan mağaza hangisidir ve toplam kazancı nedir?

In [51]:
# Gereksiz 'last_update' sütunlarını silerek merge işlemlerini temiz yap
store_df = store_df.drop(columns=['last_update'], errors='ignore')
staff_df = staff_df.drop(columns=['last_update'], errors='ignore')
rental_df = rental_df.drop(columns=['last_update'], errors='ignore')
payment_df = payment_df.drop(columns=['last_update'], errors='ignore')

# 1. store + staff
store_staff_df = store_df.merge(staff_df, on='store_id', how='inner')

# 2. + rental
store_staff_rental_df = store_staff_df.merge(rental_df, on='staff_id', how='inner')

# 3. + payment
full_df = store_staff_rental_df.merge(payment_df, on='rental_id', how='inner')

# 4. store_id bazında toplam amount hesapla
revenue_df = (
    full_df
    .groupby('store_id')
    .agg(total_revenue=('amount', 'sum'))
    .reset_index()
)

# 5. Azalan sırada sıralama
revenue_df = revenue_df.sort_values(by='total_revenue', ascending=False)

# 6. En çok gelir getiren store
top_store = revenue_df.head(1)

print(top_store)

   store_id  total_revenue
1         2       33881.94


### Soru 95: En az kazanç sağlayan mağaza hangisidir ve toplam kazancı nedir?

In [53]:
revenue_df = revenue_df.sort_values(by='total_revenue', ascending=True)

#  En az gelir getiren store
top_store = revenue_df.head(1)

print(top_store)

   store_id  total_revenue
0         1       33524.62


### Soru 96: En fazla sayıda farklı film kiralayan müşteri hangisidir?

In [55]:
# 1. customer ve rental'ı customer_id üzerinden birleştir
cust_rental_df = customer_df.merge(rental_df, on='customer_id', how='inner')

# 2. Her müşterinin (first_name, last_name) bazında kaç farklı inventory_id kiraladığını bul
unique_rentals_df = (
    cust_rental_df
    .groupby(['first_name', 'last_name'])['inventory_id']
    .nunique()  # DISTINCT COUNT karşılığı
    .reset_index(name='unique_rentals')
)

# 3. Azalan sırayla sırala
unique_rentals_df = unique_rentals_df.sort_values(by='unique_rentals', ascending=False)

# 4. En çok farklı film kiralayan müşteriyi al
top_customer = unique_rentals_df.head(1)

print(top_customer)

    first_name last_name  unique_rentals
175    ELEANOR      HUNT              46


### Soru 97: En az sayıda farklı film kiralayan müşteri hangisidir?

In [57]:
unique_rentals_df = unique_rentals_df.sort_values(by='unique_rentals', ascending=True)

# En az farklı film kiralayan müşteriyi al
customer = unique_rentals_df.head(1)

print(customer)

   first_name last_name  unique_rentals
71      BRIAN     WYMAN              12


### Soru 98: En fazla sayıda farklı müşteri tarafından kiralanan film hangisidir?

In [59]:
# 1. film + inventory (film_id üzerinden)
film_inventory_df = film_df.merge(inventory_df, on='film_id', how='inner')

# 2. + rental (inventory_id üzerinden)
film_rental_df = film_inventory_df.merge(rental_df, on='inventory_id', how='inner')

# 3. title bazında kaç farklı customer_id olduğunu bul
unique_customers_df = (
    film_rental_df
    .groupby('title')['customer_id']
    .nunique()  # DISTINCT COUNT işlemi
    .reset_index(name='unique_customers')
)

# 4. Azalan sırayla sırala
unique_customers_df = unique_customers_df.sort_values(by='unique_customers', ascending=False)

# 5. En çok müşteri tarafından kiralanan filmi al
top_film = unique_customers_df.head(1)

print(top_film)

                 title  unique_customers
96  BUCKET BROTHERHOOD                33


### Soru 99: En az sayıda farklı müşteri tarafından kiralanan film hangisidir?

In [61]:
# artan sırayla sırala
unique_customers_df = unique_customers_df.sort_values(by='unique_customers', ascending=True)

# En az müşteri tarafından kiralanan filmi al
film = unique_customers_df.head(1)

print(film)

              title  unique_customers
378  HARDLY ROBBERS                 4


### Soru 100: Hangi mağazada, hangi kategoride en fazla kazanç sağlanmış?

In [67]:
# Gereksiz çakışmaları önlemek için tüm last_update sütunlarını silelim
for df in [store_df, staff_df, rental_df, inventory_df, film_df, film_category_df, category_df, payment_df]:
    df.drop(columns=['last_update'], inplace=True, errors='ignore')
    
# 1. store + staff
df1 = store_df.merge(staff_df, on='store_id', how='inner')

# 2. + rental (staff_id üzerinden)
df2 = df1.merge(rental_df, on='staff_id', how='inner')

# 3. + inventory (inventory_id üzerinden)
df3 = df2.merge(inventory_df, on='inventory_id', how='inner')

# 4. + film (film_id üzerinden)
df4 = df3.merge(film_df, on='film_id', how='inner')

# 5. + film_category (film_id)
df5 = df4.merge(film_category_df, on='film_id', how='inner')

# 6. + category (category_id)
df6 = df5.merge(category_df, on='category_id', how='inner')

# 7. + payment (rental_id üzerinden)
full_df = df6.merge(payment_df, on='rental_id', how='inner')

# 8. store_id ve genre (category.name) bazında toplam gelir
revenue_df = (
    full_df
    .groupby(['store_id_x', 'name'])
    .agg(total_revenue=('amount', 'sum'))
    .reset_index()
)

# 9. genre ismini düzelt
revenue_df = revenue_df.rename(columns={'name': 'genre'})

# 10. en yüksek geliri bul
top_result = revenue_df.sort_values(by='total_revenue', ascending=False).head(1)

print(top_result)

    store_id_x   genre  total_revenue
30           2  Sports        2761.85
